## Meta Data Analysis

There are many meta data come with the images. We will analyze those information in this notebook to make sense the whole database.

In [25]:
import pandas as pd
import numpy as np
import random
import cv2
import sqlite3
import math
import matplotlib.pyplot as plt

## 1. Global Statistics

### 1.1. Chemical Annotation

`chemical_annotations.csv` contains the global information for the chemical compounds.

> Table containing metadata for many of the compounds from Broad Institute’s Chemical Biology Informatics Platform (CBIP), including (where applicable) compound names, simplified molecular-input line-entry system annotations (SMILES), MLSMR sample identifiers, and PubChem compound identifiers (CID) and substance identifiers (SID).

In [2]:
df = pd.read_csv("./data/test/meta_data/chemical_annotations.csv")
print(df.shape)
df.head(7)

(30616, 10)


,BROAD_ID,CPD_NAME,CPD_NAME_TYPE,CPD_SAMPLE_ID,DOS_LIBRARY,SOURCE_NAME,CHEMIST_NAME,VENDOR_CATALOG_ID,CPD_SMILES,USERCOMMENT
0,BRD-A56675431-001-04-0,altizide,INN,SA82748,NaN,Prestwick Chemical Inc.,NaN,Prestw-721,NS(=O)(=O)c1cc2c(NC(CSCC=C)NS2(=O)=O)cc1Cl,NaN
1,BRD-A51829654-001-01-4,"BRL-15,572",common,SA82481,NaN,Biomol International Inc.,NaN,AC-536,OC(CN1CCN(CC1)c1cccc(Cl)c1)C(c1ccccc1)c1ccccc1,NaN
2,BRD-K04046242-001-03-6,equilin,primary-common,SA82922,NaN,Prestwick Chemical Inc.,NaN,Prestw-850,C[C@]12CC[C@H]3C(=CCc4cc(O)ccc34)[C@@H]1CCC2=O,NaN
3,BRD-K16508793-001-01-8,diazepam,INN,SA59660,NaN,MicroSource Discovery Systems Inc.,NaN,1900003,CN1c2ccc(Cl)cc2C(=NCC1=O)c1ccccc1,NaN
4,BRD-K09397065-001-01-6,SR 57227A,to-be-curated,SA82504,NaN,Biomol International Inc.,NaN,AC-561,NC1CCN(CC1)c1cccc(Cl)n1,NaN
5,BRD-K11927976-050-01-1,ER-27319,to-be-curated,SA792875,NaN,Tocris Bioscience,NaN,2471,Cc1ccc2c(c1C)n(CCCN)c1ccccc1c2=O,NaN
6,BRD-K14282469-001-01-5,PAPP,primary-common,SA82523,NaN,Biomol International Inc.,NaN,AC-846,Nc1ccc(CCN2CCN(CC2)c2cccc(c2)C(F)(F)F)cc1,NaN


### 1.2. Image Name Encoding

Each plate has an unique 5-digit plate number. The Cell Image Library actually has more plates than the meta data provided in Giga database.

Each plate has 5 channels using dyes `ERSyto`, `ERSytoBleed`, `Hoechst`, `Mito` and `Ph_golgi`. For example, the dir name of plate 24278 is:
- `24278-ERSyto`
- `24278-ERSytoBleed`
- `24278-Hoechst`
- `24278-Mito`
- `24278-Ph_golgi`

Those dyes have different names in the literature and some part of the literature, below is a mapping of those names:

|Dye| Alternative| Position|
|:---:|:---:|:---:|
|ERSyto|ER| Endoplasmic reticulum|
|ERSytoBleed| RNA| RNA|
|Hoechst| DNA| Nucleus|
|Mito| Mito| Mitochondria|
|Ph_golgi| AGP| plasma membrane|


Each image is encoded in 16-bit tiff format, with name like `cdp2bioactives_a01_s1_w2edcec6dc-b1e3-4ffc-80da-9b049a89447b.tif`.

- `a01` is the well index
- `s1` is the site index (depth of field)
- I suppose `w2edcec6dc-b1e3-4ffc-80da-9b049a89447b` is just a unique identifier for each image.
- Same well in different channel has the same name (i.e. `cdp2bioactives_a01_s1_`) before the long identifier.


### 1.3. Image Statistics

This table includes the image statistics for each plate.

- **Plate ID**: 5-digit identifier given by the ImageXpress microscope labeling the plate.
- **Num_CIL_images**: Total number of images for the plate hosted at The Cell Image Library (CIL).
- **Num_CIL_wells**: Total number of wells represented in the plate hosted at CIL which have >1 site (i.e., field of view) included.
- **Num_CIL_complete wells**: Total number of wells which have all sites included.
- **Num_CIL_sites**: Total number of sites which have >1 channel included.
- **Num_CIL_complete_sites**: Total number of sites which have all channels included.
- **Num_QC_stats**: Total number of sites for which quality control data is included.
- **Num_blurry_sites**: Total number of sites labelled as blurry/out-of-focus by the quality control workflow.
- **Num_saturated_sites**: Total number of sites labelled as containing saturation artifacts by the quality control workflow.


In [3]:
df = pd.read_csv("./data/test/meta_data/image_curation_statistics.csv")
df.head()

,PlateID,Num_CIL_images,Num_CIL_wells,Num_CIL_complete wells,Num_CIL_sites,Num_CIL_complete_sites,Num_QC_stats,Num_blurry_sites,Num_saturated_sites,Num_well_profiles
0,24277,11520,384,384,2304,2304,2304,0,75,384
1,24278,2445,82,81,489,489,489,1,6,384
2,24279,11520,384,384,2304,2304,2304,0,30,384
3,24280,11520,384,384,2304,2304,2304,0,48,383
4,24293,11520,384,384,2304,2304,2304,0,32,384


## 2. Local Statistics

For the following tables/database, each one is associated to one plate. I use plate 24278 for the demonstration.

### 2.1. Quality Control

This table gives the imaging quality statistics in well and site level, but it has no channel information.

Each field of view is assessed for the presence of 2 artifacts (focal blur and saturated objects), and assigned a label of 1 if present and 0 if not. 

In [4]:
df = pd.read_csv("./data/test/meta_data/quality_control/qc.csv")
df.head(7)

,Image_Metadata_Plate,Image_Metadata_Well,Image_Metadata_Site,Image_Metadata_isSaturated,Image_Metadata_isBlurry
0,24278,a20,1,0,0
1,24278,a20,2,0,0
2,24278,a20,3,0,0
3,24278,a20,4,0,0
4,24278,a20,5,0,0
5,24278,a20,6,0,0
6,24278,a21,1,0,0


### 2.2. Mean Well Profiles

Per-well averages of each extracted morphological feature computed across the cells.

For each well/site, there are about 1800 average morphology features.

**The compound information is also recorded in this table**. The column `Metadata_broad_sample` is a foreign key of Chemical Annotation table (`BROAD_ID`).

In [5]:
df = pd.read_csv("./data/test/meta_data/profiles/mean_well_profiles.csv")
print(df.shape)
df.head(10)

(384, 1800)


,Metadata_Plate,Metadata_Well,Metadata_Assay_Plate_Barcode,Metadata_Plate_Map_Name,Metadata_well_position,Metadata_ASSAY_WELL_ROLE,Metadata_broad_sample,Metadata_mmoles_per_liter,Metadata_solvent,Metadata_pert_id,...,Nuclei_Texture_Variance_DNA_5_0,Nuclei_Texture_Variance_ER_10_0,Nuclei_Texture_Variance_ER_3_0,Nuclei_Texture_Variance_ER_5_0,Nuclei_Texture_Variance_Mito_10_0,Nuclei_Texture_Variance_Mito_3_0,Nuclei_Texture_Variance_Mito_5_0,Nuclei_Texture_Variance_RNA_10_0,Nuclei_Texture_Variance_RNA_3_0,Nuclei_Texture_Variance_RNA_5_0
0,24278,a01,24278,H-BIOA-007-3,a01,treated,BRD-K78364995-236-03-5,2.090060,DMSO,BRD-K78364995,...,3.065597,1.525995,1.596880,1.557685,1.633298,1.627728,1.618267,2.582515,2.474853,2.528092
1,24278,a02,24278,H-BIOA-007-3,a02,treated,BRD-K78414110-001-02-8,5.000000,DMSO,BRD-K78414110,...,2.947674,1.595292,1.681501,1.639656,1.438692,1.497101,1.473400,2.402188,2.443030,2.527604
2,24278,a03,24278,H-BIOA-007-3,a03,treated,BRD-K78485176-001-02-9,5.000000,DMSO,BRD-K78485176,...,2.959839,1.573649,1.641233,1.630217,1.470714,1.545823,1.499599,2.400266,2.457235,2.536708
3,24278,a04,24278,H-BIOA-007-3,a04,treated,BRD-K78496197-001-01-3,5.000000,DMSO,BRD-K78496197,...,3.323827,1.627224,1.800667,1.699291,1.419890,1.578673,1.528876,2.601877,2.439709,2.506089
4,24278,a05,24278,H-BIOA-007-3,a05,treated,BRD-K78599730-001-02-6,5.000000,DMSO,BRD-K78599730,...,3.344600,1.772216,1.913539,1.881990,1.292479,1.424874,1.378594,2.433934,2.356783,2.422295
5,24278,a06,24278,H-BIOA-007-3,a06,treated,BRD-K78612426-001-02-6,5.000000,DMSO,BRD-K78612426,...,3.499219,1.746578,1.896714,1.836252,1.604872,1.700492,1.687575,2.498050,2.398161,2.474145
6,24278,a07,24278,H-BIOA-007-3,a07,treated,BRD-K78633253-001-01-2,5.000000,DMSO,BRD-K78633253,...,3.715294,1.823429,2.092295,2.010445,1.585145,1.758397,1.675881,2.509877,2.409396,2.488828
7,24278,a08,24278,H-BIOA-007-3,a08,treated,BRD-K78637815-001-01-4,5.000000,DMSO,BRD-K78637815,...,3.360001,1.693696,1.795996,1.783097,1.587473,1.714071,1.649360,2.390896,2.383596,2.445147
8,24278,a09,24278,H-BIOA-007-3,a09,treated,BRD-K78643075-001-03-3,2.379167,DMSO,BRD-K78643075,...,3.559220,1.857709,1.973541,1.909170,1.654707,1.760128,1.705003,2.320583,2.282689,2.343038
9,24278,a10,24278,H-BIOA-007-3,a10,treated,BRD-K78692225-001-11-2,5.000000,DMSO,BRD-K78692225,...,3.543376,1.810797,1.847711,1.791665,1.531595,1.699469,1.603155,2.381528,2.369899,2.450524


In [6]:
chem_df = pd.read_csv("./data/test/meta_data/chemical_annotations.csv")
index = list(chem_df['BROAD_ID']).index(df['Metadata_broad_sample'][0])
chem_df.iloc[index]

BROAD_ID                                        BRD-K78364995-236-03-5
CPD_NAME                                                    cefotaxime
CPD_NAME_TYPE                                                      INN
CPD_SAMPLE_ID                                                  SA83374
DOS_LIBRARY                                                        NaN
SOURCE_NAME                                    Prestwick Chemical Inc.
CHEMIST_NAME                                                       NaN
VENDOR_CATALOG_ID                                           Prestw-139
CPD_SMILES           CO\N=C(/C(=O)N[C@H]1[C@H]2SCC(COC(C)=O)=C(N2C1...
USERCOMMENT                                                        NaN
Name: 650, dtype: object

For example, we find that Well a01 in Plate 24278 has compound "cefotaxime" below.

## 2.3 Extracted Feature

A SQLite database comprising 4 tables (a) 1 per-image cellular statistic (e.g., cell count), (b) 3 per-cell cell tables, measuring size, shape, intensity, textural, and adjacency statistics for the nuclei, cytoplasm, and cell body. 

1. Cells Table
2. Cytoplasm Table
3. Image Table
4. Nuclei Table

`Image` Table is the entry table with a combined primary key `Image_Metadata_Plate`, `Image_Metadata_Site`, and `Image_Metadata_Well`. Other tables `Cells`, `Cytoplasm` and `Nuclei` can be queried using a combined primary key `TableNumber`, `ImageNumber` and `ObjectNumber`. The first two keys can be queried from `Image` table, and `ObjectNumber` is an integer bounded in the count of cell/cytoplasm/nuclei.

In [7]:
conn = sqlite3.connect('./data/test/meta_data/extracted_features/24278.sqlite')
c = conn.cursor()

In [8]:
# Display all column names in Image table
# cur = c.execute("SELECT * FROM Nuclei")
# [tuple[0] for tuple in cur.description]

All features are listed below:

<details><summary>`Image` Table</summary>
<p>
'TableNumber',
 'Image_Count_Cells',
 'Image_Count_Cytoplasm',
 'Image_Count_Nuclei',
 'Image_ExecutionTime_01LoadData',
 'Image_ExecutionTime_02CorrectIlluminationApply',
 'Image_ExecutionTime_03MeasureImageQuality',
 'Image_ExecutionTime_04MeasureImageQuality',
 'Image_ExecutionTime_06IdentifyPrimaryObjects',
 'Image_ExecutionTime_07IdentifySecondaryObjects',
 'Image_ExecutionTime_08IdentifyTertiaryObjects',
 'Image_ExecutionTime_09MeasureCorrelation',
 'Image_ExecutionTime_10MeasureGranularity',
 'Image_ExecutionTime_11MeasureObjectIntensity',
 'Image_ExecutionTime_12MeasureObjectNeighbors',
 'Image_ExecutionTime_13MeasureObjectNeighbors',
 'Image_ExecutionTime_14MeasureObjectNeighbors',
 'Image_ExecutionTime_15MeasureObjectIntensityDistribution',
 'Image_ExecutionTime_16MeasureObjectSizeShape',
 'Image_ExecutionTime_17MeasureTexture',
 'Image_ExecutionTime_18OverlayOutlines',
 'Image_ExecutionTime_19OverlayOutlines',
 'Image_ExecutionTime_20SaveImages',
 'Image_ExecutionTime_21SaveImages',
 'Image_FileName_CellOutlines',
 'Image_FileName_IllumAGP',
 'Image_FileName_IllumDNA',
 'Image_FileName_IllumER',
 'Image_FileName_IllumMito',
 'Image_FileName_IllumRNA',
 'Image_FileName_NucleiOutlines',
 'Image_FileName_OrigAGP',
 'Image_FileName_OrigDNA',
 'Image_FileName_OrigER',
 'Image_FileName_OrigMito',
 'Image_FileName_OrigRNA',
 'Image_Granularity_10_AGP',
 'Image_Granularity_10_DNA',
 'Image_Granularity_10_ER',
 'Image_Granularity_10_Mito',
 'Image_Granularity_10_RNA',
 'Image_Granularity_11_AGP',
 'Image_Granularity_11_DNA',
 'Image_Granularity_11_ER',
 'Image_Granularity_11_Mito',
 'Image_Granularity_11_RNA',
 'Image_Granularity_12_AGP',
 'Image_Granularity_12_DNA',
 'Image_Granularity_12_ER',
 'Image_Granularity_12_Mito',
 'Image_Granularity_12_RNA',
 'Image_Granularity_13_AGP',
 'Image_Granularity_13_DNA',
 'Image_Granularity_13_ER',
 'Image_Granularity_13_Mito',
 'Image_Granularity_13_RNA',
 'Image_Granularity_14_AGP',
 'Image_Granularity_14_DNA',
 'Image_Granularity_14_ER',
 'Image_Granularity_14_Mito',
 'Image_Granularity_14_RNA',
 'Image_Granularity_15_AGP',
 'Image_Granularity_15_DNA',
 'Image_Granularity_15_ER',
 'Image_Granularity_15_Mito',
 'Image_Granularity_15_RNA',
 'Image_Granularity_16_AGP',
 'Image_Granularity_16_DNA',
 'Image_Granularity_16_ER',
 'Image_Granularity_16_Mito',
 'Image_Granularity_16_RNA',
 'Image_Granularity_1_AGP',
 'Image_Granularity_1_DNA',
 'Image_Granularity_1_ER',
 'Image_Granularity_1_Mito',
 'Image_Granularity_1_RNA',
 'Image_Granularity_2_AGP',
 'Image_Granularity_2_DNA',
 'Image_Granularity_2_ER',
 'Image_Granularity_2_Mito',
 'Image_Granularity_2_RNA',
 'Image_Granularity_3_AGP',
 'Image_Granularity_3_DNA',
 'Image_Granularity_3_ER',
 'Image_Granularity_3_Mito',
 'Image_Granularity_3_RNA',
 'Image_Granularity_4_AGP',
 'Image_Granularity_4_DNA',
 'Image_Granularity_4_ER',
 'Image_Granularity_4_Mito',
 'Image_Granularity_4_RNA',
 'Image_Granularity_5_AGP',
 'Image_Granularity_5_DNA',
 'Image_Granularity_5_ER',
 'Image_Granularity_5_Mito',
 'Image_Granularity_5_RNA',
 'Image_Granularity_6_AGP',
 'Image_Granularity_6_DNA',
 'Image_Granularity_6_ER',
 'Image_Granularity_6_Mito',
 'Image_Granularity_6_RNA',
 'Image_Granularity_7_AGP',
 'Image_Granularity_7_DNA',
 'Image_Granularity_7_ER',
 'Image_Granularity_7_Mito',
 'Image_Granularity_7_RNA',
 'Image_Granularity_8_AGP',
 'Image_Granularity_8_DNA',
 'Image_Granularity_8_ER',
 'Image_Granularity_8_Mito',
 'Image_Granularity_8_RNA',
 'Image_Granularity_9_AGP',
 'Image_Granularity_9_DNA',
 'Image_Granularity_9_ER',
 'Image_Granularity_9_Mito',
 'Image_Granularity_9_RNA',
 'Image_Group_Index',
 'Image_Group_Number',
 'Image_Height_IllumAGP',
 'Image_Height_IllumDNA',
 'Image_Height_IllumER',
 'Image_Height_IllumMito',
 'Image_Height_IllumRNA',
 'Image_Height_OrigAGP',
 'Image_Height_OrigDNA',
 'Image_Height_OrigER',
 'Image_Height_OrigMito',
 'Image_Height_OrigRNA',
 'ImageNumber',
 'Image_ImageQuality_Correlation_IllumAGP_10',
 'Image_ImageQuality_Correlation_IllumAGP_2',
 'Image_ImageQuality_Correlation_IllumAGP_20',
 'Image_ImageQuality_Correlation_IllumAGP_5',
 'Image_ImageQuality_Correlation_IllumDNA_10',
 'Image_ImageQuality_Correlation_IllumDNA_2',
 'Image_ImageQuality_Correlation_IllumDNA_20',
 'Image_ImageQuality_Correlation_IllumDNA_5',
 'Image_ImageQuality_Correlation_IllumER_10',
 'Image_ImageQuality_Correlation_IllumER_2',
 'Image_ImageQuality_Correlation_IllumER_20',
 'Image_ImageQuality_Correlation_IllumER_5',
 'Image_ImageQuality_Correlation_IllumMito_10',
 'Image_ImageQuality_Correlation_IllumMito_2',
 'Image_ImageQuality_Correlation_IllumMito_20',
 'Image_ImageQuality_Correlation_IllumMito_5',
 'Image_ImageQuality_Correlation_IllumRNA_10',
 'Image_ImageQuality_Correlation_IllumRNA_2',
 'Image_ImageQuality_Correlation_IllumRNA_20',
 'Image_ImageQuality_Correlation_IllumRNA_5',
 'Image_ImageQuality_Correlation_OrigAGP_10',
 'Image_ImageQuality_Correlation_OrigAGP_2',
 'Image_ImageQuality_Correlation_OrigAGP_20',
 'Image_ImageQuality_Correlation_OrigAGP_5',
 'Image_ImageQuality_Correlation_OrigDNA_10',
 'Image_ImageQuality_Correlation_OrigDNA_2',
 'Image_ImageQuality_Correlation_OrigDNA_20',
 'Image_ImageQuality_Correlation_OrigDNA_5',
 'Image_ImageQuality_Correlation_OrigER_10',
 'Image_ImageQuality_Correlation_OrigER_2',
 'Image_ImageQuality_Correlation_OrigER_20',
 'Image_ImageQuality_Correlation_OrigER_5',
 'Image_ImageQuality_Correlation_OrigMito_10',
 'Image_ImageQuality_Correlation_OrigMito_2',
 'Image_ImageQuality_Correlation_OrigMito_20',
 'Image_ImageQuality_Correlation_OrigMito_5',
 'Image_ImageQuality_Correlation_OrigRNA_10',
 'Image_ImageQuality_Correlation_OrigRNA_2',
 'Image_ImageQuality_Correlation_OrigRNA_20',
 'Image_ImageQuality_Correlation_OrigRNA_5',
 'Image_ImageQuality_FocusScore_IllumAGP',
 'Image_ImageQuality_FocusScore_IllumDNA',
 'Image_ImageQuality_FocusScore_IllumER',
 'Image_ImageQuality_FocusScore_IllumMito',
 'Image_ImageQuality_FocusScore_IllumRNA',
 'Image_ImageQuality_FocusScore_OrigAGP',
 'Image_ImageQuality_FocusScore_OrigDNA',
 'Image_ImageQuality_FocusScore_OrigER',
 'Image_ImageQuality_FocusScore_OrigMito',
 'Image_ImageQuality_FocusScore_OrigRNA',
 'Image_ImageQuality_LocalFocusScore_IllumAGP_10',
 'Image_ImageQuality_LocalFocusScore_IllumAGP_2',
 'Image_ImageQuality_LocalFocusScore_IllumAGP_20',
 'Image_ImageQuality_LocalFocusScore_IllumAGP_5',
 'Image_ImageQuality_LocalFocusScore_IllumDNA_10',
 'Image_ImageQuality_LocalFocusScore_IllumDNA_2',
 'Image_ImageQuality_LocalFocusScore_IllumDNA_20',
 'Image_ImageQuality_LocalFocusScore_IllumDNA_5',
 'Image_ImageQuality_LocalFocusScore_IllumER_10',
 'Image_ImageQuality_LocalFocusScore_IllumER_2',
 'Image_ImageQuality_LocalFocusScore_IllumER_20',
 'Image_ImageQuality_LocalFocusScore_IllumER_5',
 'Image_ImageQuality_LocalFocusScore_IllumMito_10',
 'Image_ImageQuality_LocalFocusScore_IllumMito_2',
 'Image_ImageQuality_LocalFocusScore_IllumMito_20',
 'Image_ImageQuality_LocalFocusScore_IllumMito_5',
 'Image_ImageQuality_LocalFocusScore_IllumRNA_10',
 'Image_ImageQuality_LocalFocusScore_IllumRNA_2',
 'Image_ImageQuality_LocalFocusScore_IllumRNA_20',
 'Image_ImageQuality_LocalFocusScore_IllumRNA_5',
 'Image_ImageQuality_LocalFocusScore_OrigAGP_10',
 'Image_ImageQuality_LocalFocusScore_OrigAGP_2',
 'Image_ImageQuality_LocalFocusScore_OrigAGP_20',
 'Image_ImageQuality_LocalFocusScore_OrigAGP_5',
 'Image_ImageQuality_LocalFocusScore_OrigDNA_10',
 'Image_ImageQuality_LocalFocusScore_OrigDNA_2',
 'Image_ImageQuality_LocalFocusScore_OrigDNA_20',
 'Image_ImageQuality_LocalFocusScore_OrigDNA_5',
 'Image_ImageQuality_LocalFocusScore_OrigER_10',
 'Image_ImageQuality_LocalFocusScore_OrigER_2',
 'Image_ImageQuality_LocalFocusScore_OrigER_20',
 'Image_ImageQuality_LocalFocusScore_OrigER_5',
 'Image_ImageQuality_LocalFocusScore_OrigMito_10',
 'Image_ImageQuality_LocalFocusScore_OrigMito_2',
 'Image_ImageQuality_LocalFocusScore_OrigMito_20',
 'Image_ImageQuality_LocalFocusScore_OrigMito_5',
 'Image_ImageQuality_LocalFocusScore_OrigRNA_10',
 'Image_ImageQuality_LocalFocusScore_OrigRNA_2',
 'Image_ImageQuality_LocalFocusScore_OrigRNA_20',
 'Image_ImageQuality_LocalFocusScore_OrigRNA_5',
 'Image_ImageQuality_MADIntensity_IllumAGP',
 'Image_ImageQuality_MADIntensity_IllumDNA',
 'Image_ImageQuality_MADIntensity_IllumER',
 'Image_ImageQuality_MADIntensity_IllumMito',
 'Image_ImageQuality_MADIntensity_IllumRNA',
 'Image_ImageQuality_MADIntensity_OrigAGP',
 'Image_ImageQuality_MADIntensity_OrigDNA',
 'Image_ImageQuality_MADIntensity_OrigER',
 'Image_ImageQuality_MADIntensity_OrigMito',
 'Image_ImageQuality_MADIntensity_OrigRNA',
 'Image_ImageQuality_MaxIntensity_IllumAGP',
 'Image_ImageQuality_MaxIntensity_IllumDNA',
 'Image_ImageQuality_MaxIntensity_IllumER',
 'Image_ImageQuality_MaxIntensity_IllumMito',
 'Image_ImageQuality_MaxIntensity_IllumRNA',
 'Image_ImageQuality_MaxIntensity_OrigAGP',
 'Image_ImageQuality_MaxIntensity_OrigDNA',
 'Image_ImageQuality_MaxIntensity_OrigER',
 'Image_ImageQuality_MaxIntensity_OrigMito',
 'Image_ImageQuality_MaxIntensity_OrigRNA',
 'Image_ImageQuality_MeanIntensity_IllumAGP',
 'Image_ImageQuality_MeanIntensity_IllumDNA',
 'Image_ImageQuality_MeanIntensity_IllumER',
 'Image_ImageQuality_MeanIntensity_IllumMito',
 'Image_ImageQuality_MeanIntensity_IllumRNA',
 'Image_ImageQuality_MeanIntensity_OrigAGP',
 'Image_ImageQuality_MeanIntensity_OrigDNA',
 'Image_ImageQuality_MeanIntensity_OrigER',
 'Image_ImageQuality_MeanIntensity_OrigMito',
 'Image_ImageQuality_MeanIntensity_OrigRNA',
 'Image_ImageQuality_MedianIntensity_IllumAGP',
 'Image_ImageQuality_MedianIntensity_IllumDNA',
 'Image_ImageQuality_MedianIntensity_IllumER',
 'Image_ImageQuality_MedianIntensity_IllumMito',
 'Image_ImageQuality_MedianIntensity_IllumRNA',
 'Image_ImageQuality_MedianIntensity_OrigAGP',
 'Image_ImageQuality_MedianIntensity_OrigDNA',
 'Image_ImageQuality_MedianIntensity_OrigER',
 'Image_ImageQuality_MedianIntensity_OrigMito',
 'Image_ImageQuality_MedianIntensity_OrigRNA',
 'Image_ImageQuality_MinIntensity_IllumAGP',
 'Image_ImageQuality_MinIntensity_IllumDNA',
 'Image_ImageQuality_MinIntensity_IllumER',
 'Image_ImageQuality_MinIntensity_IllumMito',
 'Image_ImageQuality_MinIntensity_IllumRNA',
 'Image_ImageQuality_MinIntensity_OrigAGP',
 'Image_ImageQuality_MinIntensity_OrigDNA',
 'Image_ImageQuality_MinIntensity_OrigER',
 'Image_ImageQuality_MinIntensity_OrigMito',
 'Image_ImageQuality_MinIntensity_OrigRNA',
 'Image_ImageQuality_PercentMaximal_IllumAGP',
 'Image_ImageQuality_PercentMaximal_IllumDNA',
 'Image_ImageQuality_PercentMaximal_IllumER',
 'Image_ImageQuality_PercentMaximal_IllumMito',
 'Image_ImageQuality_PercentMaximal_IllumRNA',
 'Image_ImageQuality_PercentMaximal_OrigAGP',
 'Image_ImageQuality_PercentMaximal_OrigDNA',
 'Image_ImageQuality_PercentMaximal_OrigER',
 'Image_ImageQuality_PercentMaximal_OrigMito',
 'Image_ImageQuality_PercentMaximal_OrigRNA',
 'Image_ImageQuality_PercentMinimal_IllumAGP',
 'Image_ImageQuality_PercentMinimal_IllumDNA',
 'Image_ImageQuality_PercentMinimal_IllumER',
 'Image_ImageQuality_PercentMinimal_IllumMito',
 'Image_ImageQuality_PercentMinimal_IllumRNA',
 'Image_ImageQuality_PercentMinimal_OrigAGP',
 'Image_ImageQuality_PercentMinimal_OrigDNA',
 'Image_ImageQuality_PercentMinimal_OrigER',
 'Image_ImageQuality_PercentMinimal_OrigMito',
 'Image_ImageQuality_PercentMinimal_OrigRNA',
 'Image_ImageQuality_PowerLogLogSlope_IllumAGP',
 'Image_ImageQuality_PowerLogLogSlope_IllumDNA',
 'Image_ImageQuality_PowerLogLogSlope_IllumER',
 'Image_ImageQuality_PowerLogLogSlope_IllumMito',
 'Image_ImageQuality_PowerLogLogSlope_IllumRNA',
 'Image_ImageQuality_PowerLogLogSlope_OrigAGP',
 'Image_ImageQuality_PowerLogLogSlope_OrigDNA',
 'Image_ImageQuality_PowerLogLogSlope_OrigER',
 'Image_ImageQuality_PowerLogLogSlope_OrigMito',
 'Image_ImageQuality_PowerLogLogSlope_OrigRNA',
 'Image_ImageQuality_Scaling_IllumAGP',
 'Image_ImageQuality_Scaling_IllumDNA',
 'Image_ImageQuality_Scaling_IllumER',
 'Image_ImageQuality_Scaling_IllumMito',
 'Image_ImageQuality_Scaling_IllumRNA',
 'Image_ImageQuality_Scaling_OrigAGP',
 'Image_ImageQuality_Scaling_OrigDNA',
 'Image_ImageQuality_Scaling_OrigER',
 'Image_ImageQuality_Scaling_OrigMito',
 'Image_ImageQuality_Scaling_OrigRNA',
 'Image_ImageQuality_StdIntensity_IllumAGP',
 'Image_ImageQuality_StdIntensity_IllumDNA',
 'Image_ImageQuality_StdIntensity_IllumER',
 'Image_ImageQuality_StdIntensity_IllumMito',
 'Image_ImageQuality_StdIntensity_IllumRNA',
 'Image_ImageQuality_StdIntensity_OrigAGP',
 'Image_ImageQuality_StdIntensity_OrigDNA',
 'Image_ImageQuality_StdIntensity_OrigER',
 'Image_ImageQuality_StdIntensity_OrigMito',
 'Image_ImageQuality_StdIntensity_OrigRNA',
 'Image_ImageQuality_ThresholdOtsu_OrigDNA_2W',
 'Image_ImageQuality_ThresholdOtsu_OrigRNA_3FW',
 'Image_ImageQuality_TotalArea_IllumAGP',
 'Image_ImageQuality_TotalArea_IllumDNA',
 'Image_ImageQuality_TotalArea_IllumER',
 'Image_ImageQuality_TotalArea_IllumMito',
 'Image_ImageQuality_TotalArea_IllumRNA',
 'Image_ImageQuality_TotalArea_OrigAGP',
 'Image_ImageQuality_TotalArea_OrigDNA',
 'Image_ImageQuality_TotalArea_OrigER',
 'Image_ImageQuality_TotalArea_OrigMito',
 'Image_ImageQuality_TotalArea_OrigRNA',
 'Image_ImageQuality_TotalIntensity_IllumAGP',
 'Image_ImageQuality_TotalIntensity_IllumDNA',
 'Image_ImageQuality_TotalIntensity_IllumER',
 'Image_ImageQuality_TotalIntensity_IllumMito',
 'Image_ImageQuality_TotalIntensity_IllumRNA',
 'Image_ImageQuality_TotalIntensity_OrigAGP',
 'Image_ImageQuality_TotalIntensity_OrigDNA',
 'Image_ImageQuality_TotalIntensity_OrigER',
 'Image_ImageQuality_TotalIntensity_OrigMito',
 'Image_ImageQuality_TotalIntensity_OrigRNA',
 'Image_MD5Digest_IllumAGP',
 'Image_MD5Digest_IllumDNA',
 'Image_MD5Digest_IllumER',
 'Image_MD5Digest_IllumMito',
 'Image_MD5Digest_IllumRNA',
 'Image_MD5Digest_OrigAGP',
 'Image_MD5Digest_OrigDNA',
 'Image_MD5Digest_OrigER',
 'Image_MD5Digest_OrigMito',
 'Image_MD5Digest_OrigRNA',
 'Image_Metadata_Plate',
 'Image_Metadata_Site',
 'Image_Metadata_Well',
 'Image_ModuleError_01LoadData',
 'Image_ModuleError_02CorrectIlluminationApply',
 'Image_ModuleError_03MeasureImageQuality',
 'Image_ModuleError_04MeasureImageQuality',
 'Image_ModuleError_06IdentifyPrimaryObjects',
 'Image_ModuleError_07IdentifySecondaryObjects',
 'Image_ModuleError_08IdentifyTertiaryObjects',
 'Image_ModuleError_09MeasureCorrelation',
 'Image_ModuleError_10MeasureGranularity',
 'Image_ModuleError_11MeasureObjectIntensity',
 'Image_ModuleError_12MeasureObjectNeighbors',
 'Image_ModuleError_13MeasureObjectNeighbors',
 'Image_ModuleError_14MeasureObjectNeighbors',
 'Image_ModuleError_15MeasureObjectIntensityDistribution',
 'Image_ModuleError_16MeasureObjectSizeShape',
 'Image_ModuleError_17MeasureTexture',
 'Image_ModuleError_18OverlayOutlines',
 'Image_ModuleError_19OverlayOutlines',
 'Image_ModuleError_20SaveImages',
 'Image_ModuleError_21SaveImages',
 'Image_PathName_CellOutlines',
 'Image_PathName_IllumAGP',
 'Image_PathName_IllumDNA',
 'Image_PathName_IllumER',
 'Image_PathName_IllumMito',
 'Image_PathName_IllumRNA',
 'Image_PathName_NucleiOutlines',
 'Image_PathName_OrigAGP',
 'Image_PathName_OrigDNA',
 'Image_PathName_OrigER',
 'Image_PathName_OrigMito',
 'Image_PathName_OrigRNA',
 'Image_Scaling_IllumAGP',
 'Image_Scaling_IllumDNA',
 'Image_Scaling_IllumER',
 'Image_Scaling_IllumMito',
 'Image_Scaling_IllumRNA',
 'Image_Scaling_OrigAGP',
 'Image_Scaling_OrigDNA',
 'Image_Scaling_OrigER',
 'Image_Scaling_OrigMito',
 'Image_Scaling_OrigRNA',
 'Image_Texture_AngularSecondMoment_AGP_10_0',
 'Image_Texture_AngularSecondMoment_AGP_3_0',
 'Image_Texture_AngularSecondMoment_AGP_5_0',
 'Image_Texture_AngularSecondMoment_DNA_10_0',
 'Image_Texture_AngularSecondMoment_DNA_3_0',
 'Image_Texture_AngularSecondMoment_DNA_5_0',
 'Image_Texture_AngularSecondMoment_ER_10_0',
 'Image_Texture_AngularSecondMoment_ER_3_0',
 'Image_Texture_AngularSecondMoment_ER_5_0',
 'Image_Texture_AngularSecondMoment_Mito_10_0',
 'Image_Texture_AngularSecondMoment_Mito_3_0',
 'Image_Texture_AngularSecondMoment_Mito_5_0',
 'Image_Texture_AngularSecondMoment_RNA_10_0',
 'Image_Texture_AngularSecondMoment_RNA_3_0',
 'Image_Texture_AngularSecondMoment_RNA_5_0',
 'Image_Texture_Contrast_AGP_10_0',
 'Image_Texture_Contrast_AGP_3_0',
 'Image_Texture_Contrast_AGP_5_0',
 'Image_Texture_Contrast_DNA_10_0',
 'Image_Texture_Contrast_DNA_3_0',
 'Image_Texture_Contrast_DNA_5_0',
 'Image_Texture_Contrast_ER_10_0',
 'Image_Texture_Contrast_ER_3_0',
 'Image_Texture_Contrast_ER_5_0',
 'Image_Texture_Contrast_Mito_10_0',
 'Image_Texture_Contrast_Mito_3_0',
 'Image_Texture_Contrast_Mito_5_0',
 'Image_Texture_Contrast_RNA_10_0',
 'Image_Texture_Contrast_RNA_3_0',
 'Image_Texture_Contrast_RNA_5_0',
 'Image_Texture_Correlation_AGP_10_0',
 'Image_Texture_Correlation_AGP_3_0',
 'Image_Texture_Correlation_AGP_5_0',
 'Image_Texture_Correlation_DNA_10_0',
 'Image_Texture_Correlation_DNA_3_0',
 'Image_Texture_Correlation_DNA_5_0',
 'Image_Texture_Correlation_ER_10_0',
 'Image_Texture_Correlation_ER_3_0',
 'Image_Texture_Correlation_ER_5_0',
 'Image_Texture_Correlation_Mito_10_0',
 'Image_Texture_Correlation_Mito_3_0',
 'Image_Texture_Correlation_Mito_5_0',
 'Image_Texture_Correlation_RNA_10_0',
 'Image_Texture_Correlation_RNA_3_0',
 'Image_Texture_Correlation_RNA_5_0',
 'Image_Texture_DifferenceEntropy_AGP_10_0',
 'Image_Texture_DifferenceEntropy_AGP_3_0',
 'Image_Texture_DifferenceEntropy_AGP_5_0',
 'Image_Texture_DifferenceEntropy_DNA_10_0',
 'Image_Texture_DifferenceEntropy_DNA_3_0',
 'Image_Texture_DifferenceEntropy_DNA_5_0',
 'Image_Texture_DifferenceEntropy_ER_10_0',
 'Image_Texture_DifferenceEntropy_ER_3_0',
 'Image_Texture_DifferenceEntropy_ER_5_0',
 'Image_Texture_DifferenceEntropy_Mito_10_0',
 'Image_Texture_DifferenceEntropy_Mito_3_0',
 'Image_Texture_DifferenceEntropy_Mito_5_0',
 'Image_Texture_DifferenceEntropy_RNA_10_0',
 'Image_Texture_DifferenceEntropy_RNA_3_0',
 'Image_Texture_DifferenceEntropy_RNA_5_0',
 'Image_Texture_DifferenceVariance_AGP_10_0',
 'Image_Texture_DifferenceVariance_AGP_3_0',
 'Image_Texture_DifferenceVariance_AGP_5_0',
 'Image_Texture_DifferenceVariance_DNA_10_0',
 'Image_Texture_DifferenceVariance_DNA_3_0',
 'Image_Texture_DifferenceVariance_DNA_5_0',
 'Image_Texture_DifferenceVariance_ER_10_0',
 'Image_Texture_DifferenceVariance_ER_3_0',
 'Image_Texture_DifferenceVariance_ER_5_0',
 'Image_Texture_DifferenceVariance_Mito_10_0',
 'Image_Texture_DifferenceVariance_Mito_3_0',
 'Image_Texture_DifferenceVariance_Mito_5_0',
 'Image_Texture_DifferenceVariance_RNA_10_0',
 'Image_Texture_DifferenceVariance_RNA_3_0',
 'Image_Texture_DifferenceVariance_RNA_5_0',
 'Image_Texture_Entropy_AGP_10_0',
 'Image_Texture_Entropy_AGP_3_0',
 'Image_Texture_Entropy_AGP_5_0',
 'Image_Texture_Entropy_DNA_10_0',
 'Image_Texture_Entropy_DNA_3_0',
 'Image_Texture_Entropy_DNA_5_0',
 'Image_Texture_Entropy_ER_10_0',
 'Image_Texture_Entropy_ER_3_0',
 'Image_Texture_Entropy_ER_5_0',
 'Image_Texture_Entropy_Mito_10_0',
 'Image_Texture_Entropy_Mito_3_0',
 'Image_Texture_Entropy_Mito_5_0',
 'Image_Texture_Entropy_RNA_10_0',
 'Image_Texture_Entropy_RNA_3_0',
 'Image_Texture_Entropy_RNA_5_0',
 'Image_Texture_Gabor_AGP_10',
 'Image_Texture_Gabor_AGP_3',
 'Image_Texture_Gabor_AGP_5',
 'Image_Texture_Gabor_DNA_10',
 'Image_Texture_Gabor_DNA_3',
 'Image_Texture_Gabor_DNA_5',
 'Image_Texture_Gabor_ER_10',
 'Image_Texture_Gabor_ER_3',
 'Image_Texture_Gabor_ER_5',
 'Image_Texture_Gabor_Mito_10',
 'Image_Texture_Gabor_Mito_3',
 'Image_Texture_Gabor_Mito_5',
 'Image_Texture_Gabor_RNA_10',
 'Image_Texture_Gabor_RNA_3',
 'Image_Texture_Gabor_RNA_5',
 'Image_Texture_InfoMeas1_AGP_10_0',
 'Image_Texture_InfoMeas1_AGP_3_0',
 'Image_Texture_InfoMeas1_AGP_5_0',
 'Image_Texture_InfoMeas1_DNA_10_0',
 'Image_Texture_InfoMeas1_DNA_3_0',
 'Image_Texture_InfoMeas1_DNA_5_0',
 'Image_Texture_InfoMeas1_ER_10_0',
 'Image_Texture_InfoMeas1_ER_3_0',
 'Image_Texture_InfoMeas1_ER_5_0',
 'Image_Texture_InfoMeas1_Mito_10_0',
 'Image_Texture_InfoMeas1_Mito_3_0',
 'Image_Texture_InfoMeas1_Mito_5_0',
 'Image_Texture_InfoMeas1_RNA_10_0',
 'Image_Texture_InfoMeas1_RNA_3_0',
 'Image_Texture_InfoMeas1_RNA_5_0',
 'Image_Texture_InfoMeas2_AGP_10_0',
 'Image_Texture_InfoMeas2_AGP_3_0',
 'Image_Texture_InfoMeas2_AGP_5_0',
 'Image_Texture_InfoMeas2_DNA_10_0',
 'Image_Texture_InfoMeas2_DNA_3_0',
 'Image_Texture_InfoMeas2_DNA_5_0',
 'Image_Texture_InfoMeas2_ER_10_0',
 'Image_Texture_InfoMeas2_ER_3_0',
 'Image_Texture_InfoMeas2_ER_5_0',
 'Image_Texture_InfoMeas2_Mito_10_0',
 'Image_Texture_InfoMeas2_Mito_3_0',
 'Image_Texture_InfoMeas2_Mito_5_0',
 'Image_Texture_InfoMeas2_RNA_10_0',
 'Image_Texture_InfoMeas2_RNA_3_0',
 'Image_Texture_InfoMeas2_RNA_5_0',
 'Image_Texture_InverseDifferenceMoment_AGP_10_0',
 'Image_Texture_InverseDifferenceMoment_AGP_3_0',
 'Image_Texture_InverseDifferenceMoment_AGP_5_0',
 'Image_Texture_InverseDifferenceMoment_DNA_10_0',
 'Image_Texture_InverseDifferenceMoment_DNA_3_0',
 'Image_Texture_InverseDifferenceMoment_DNA_5_0',
 'Image_Texture_InverseDifferenceMoment_ER_10_0',
 'Image_Texture_InverseDifferenceMoment_ER_3_0',
 'Image_Texture_InverseDifferenceMoment_ER_5_0',
 'Image_Texture_InverseDifferenceMoment_Mito_10_0',
 'Image_Texture_InverseDifferenceMoment_Mito_3_0',
 'Image_Texture_InverseDifferenceMoment_Mito_5_0',
 'Image_Texture_InverseDifferenceMoment_RNA_10_0',
 'Image_Texture_InverseDifferenceMoment_RNA_3_0',
 'Image_Texture_InverseDifferenceMoment_RNA_5_0',
 'Image_Texture_SumAverage_AGP_10_0',
 'Image_Texture_SumAverage_AGP_3_0',
 'Image_Texture_SumAverage_AGP_5_0',
 'Image_Texture_SumAverage_DNA_10_0',
 'Image_Texture_SumAverage_DNA_3_0',
 'Image_Texture_SumAverage_DNA_5_0',
 'Image_Texture_SumAverage_ER_10_0',
 'Image_Texture_SumAverage_ER_3_0',
 'Image_Texture_SumAverage_ER_5_0',
 'Image_Texture_SumAverage_Mito_10_0',
 'Image_Texture_SumAverage_Mito_3_0',
 'Image_Texture_SumAverage_Mito_5_0',
 'Image_Texture_SumAverage_RNA_10_0',
 'Image_Texture_SumAverage_RNA_3_0',
 'Image_Texture_SumAverage_RNA_5_0',
 'Image_Texture_SumEntropy_AGP_10_0',
 'Image_Texture_SumEntropy_AGP_3_0',
 'Image_Texture_SumEntropy_AGP_5_0',
 'Image_Texture_SumEntropy_DNA_10_0',
 'Image_Texture_SumEntropy_DNA_3_0',
 'Image_Texture_SumEntropy_DNA_5_0',
 'Image_Texture_SumEntropy_ER_10_0',
 'Image_Texture_SumEntropy_ER_3_0',
 'Image_Texture_SumEntropy_ER_5_0',
 'Image_Texture_SumEntropy_Mito_10_0',
 'Image_Texture_SumEntropy_Mito_3_0',
 'Image_Texture_SumEntropy_Mito_5_0',
 'Image_Texture_SumEntropy_RNA_10_0',
 'Image_Texture_SumEntropy_RNA_3_0',
 'Image_Texture_SumEntropy_RNA_5_0',
 'Image_Texture_SumVariance_AGP_10_0',
 'Image_Texture_SumVariance_AGP_3_0',
 'Image_Texture_SumVariance_AGP_5_0',
 'Image_Texture_SumVariance_DNA_10_0',
 'Image_Texture_SumVariance_DNA_3_0',
 'Image_Texture_SumVariance_DNA_5_0',
 'Image_Texture_SumVariance_ER_10_0',
 'Image_Texture_SumVariance_ER_3_0',
 'Image_Texture_SumVariance_ER_5_0',
 'Image_Texture_SumVariance_Mito_10_0',
 'Image_Texture_SumVariance_Mito_3_0',
 'Image_Texture_SumVariance_Mito_5_0',
 'Image_Texture_SumVariance_RNA_10_0',
 'Image_Texture_SumVariance_RNA_3_0',
 'Image_Texture_SumVariance_RNA_5_0',
 'Image_Texture_Variance_AGP_10_0',
 'Image_Texture_Variance_AGP_3_0',
 'Image_Texture_Variance_AGP_5_0',
 'Image_Texture_Variance_DNA_10_0',
 'Image_Texture_Variance_DNA_3_0',
 'Image_Texture_Variance_DNA_5_0',
 'Image_Texture_Variance_ER_10_0',
 'Image_Texture_Variance_ER_3_0',
 'Image_Texture_Variance_ER_5_0',
 'Image_Texture_Variance_Mito_10_0',
 'Image_Texture_Variance_Mito_3_0',
 'Image_Texture_Variance_Mito_5_0',
 'Image_Texture_Variance_RNA_10_0',
 'Image_Texture_Variance_RNA_3_0',
 'Image_Texture_Variance_RNA_5_0',
 'Image_Threshold_FinalThreshold_Cells',
 'Image_Threshold_FinalThreshold_Nuclei',
 'Image_Threshold_OrigThreshold_Cells',
 'Image_Threshold_OrigThreshold_Nuclei',
 'Image_Threshold_SumOfEntropies_Cells',
 'Image_Threshold_SumOfEntropies_Nuclei',
 'Image_Threshold_WeightedVariance_Cells',
 'Image_Threshold_WeightedVariance_Nuclei',
 'Image_URL_CellOutlines',
 'Image_URL_IllumAGP',
 'Image_URL_IllumDNA',
 'Image_URL_IllumER',
 'Image_URL_IllumMito',
 'Image_URL_IllumRNA',
 'Image_URL_NucleiOutlines',
 'Image_URL_OrigAGP',
 'Image_URL_OrigDNA',
 'Image_URL_OrigER',
 'Image_URL_OrigMito',
 'Image_URL_OrigRNA',
 'Image_Width_IllumAGP',
 'Image_Width_IllumDNA',
 'Image_Width_IllumER',
 'Image_Width_IllumMito',
 'Image_Width_IllumRNA',
 'Image_Width_OrigAGP',
 'Image_Width_OrigDNA',
 'Image_Width_OrigER',
 'Image_Width_OrigMito',
 'Image_Width_OrigRNA'
</p>
</details>

<details><summary>`Cells` Table</summary>
<p>
'TableNumber',
 'ImageNumber',
 'ObjectNumber',
 'Cells_AreaShape_Area',
 'Cells_AreaShape_Center_X',
 'Cells_AreaShape_Center_Y',
 'Cells_AreaShape_Compactness',
 'Cells_AreaShape_Eccentricity',
 'Cells_AreaShape_EulerNumber',
 'Cells_AreaShape_Extent',
 'Cells_AreaShape_FormFactor',
 'Cells_AreaShape_MajorAxisLength',
 'Cells_AreaShape_MaxFeretDiameter',
 'Cells_AreaShape_MaximumRadius',
 'Cells_AreaShape_MeanRadius',
 'Cells_AreaShape_MedianRadius',
 'Cells_AreaShape_MinFeretDiameter',
 'Cells_AreaShape_MinorAxisLength',
 'Cells_AreaShape_Orientation',
 'Cells_AreaShape_Perimeter',
 'Cells_AreaShape_Solidity',
 'Cells_AreaShape_Zernike_0_0',
 'Cells_AreaShape_Zernike_1_1',
 'Cells_AreaShape_Zernike_2_0',
 'Cells_AreaShape_Zernike_2_2',
 'Cells_AreaShape_Zernike_3_1',
 'Cells_AreaShape_Zernike_3_3',
 'Cells_AreaShape_Zernike_4_0',
 'Cells_AreaShape_Zernike_4_2',
 'Cells_AreaShape_Zernike_4_4',
 'Cells_AreaShape_Zernike_5_1',
 'Cells_AreaShape_Zernike_5_3',
 'Cells_AreaShape_Zernike_5_5',
 'Cells_AreaShape_Zernike_6_0',
 'Cells_AreaShape_Zernike_6_2',
 'Cells_AreaShape_Zernike_6_4',
 'Cells_AreaShape_Zernike_6_6',
 'Cells_AreaShape_Zernike_7_1',
 'Cells_AreaShape_Zernike_7_3',
 'Cells_AreaShape_Zernike_7_5',
 'Cells_AreaShape_Zernike_7_7',
 'Cells_AreaShape_Zernike_8_0',
 'Cells_AreaShape_Zernike_8_2',
 'Cells_AreaShape_Zernike_8_4',
 'Cells_AreaShape_Zernike_8_6',
 'Cells_AreaShape_Zernike_8_8',
 'Cells_AreaShape_Zernike_9_1',
 'Cells_AreaShape_Zernike_9_3',
 'Cells_AreaShape_Zernike_9_5',
 'Cells_AreaShape_Zernike_9_7',
 'Cells_AreaShape_Zernike_9_9',
 'Cells_Children_Cytoplasm_Count',
 'Cells_Correlation_Correlation_DNA_AGP',
 'Cells_Correlation_Correlation_DNA_ER',
 'Cells_Correlation_Correlation_DNA_Mito',
 'Cells_Correlation_Correlation_DNA_RNA',
 'Cells_Correlation_Correlation_ER_AGP',
 'Cells_Correlation_Correlation_ER_RNA',
 'Cells_Correlation_Correlation_Mito_AGP',
 'Cells_Correlation_Correlation_Mito_ER',
 'Cells_Correlation_Correlation_Mito_RNA',
 'Cells_Correlation_Correlation_RNA_AGP',
 'Cells_Correlation_Costes_AGP_DNA',
 'Cells_Correlation_Costes_AGP_ER',
 'Cells_Correlation_Costes_AGP_Mito',
 'Cells_Correlation_Costes_AGP_RNA',
 'Cells_Correlation_Costes_DNA_AGP',
 'Cells_Correlation_Costes_DNA_ER',
 'Cells_Correlation_Costes_DNA_Mito',
 'Cells_Correlation_Costes_DNA_RNA',
 'Cells_Correlation_Costes_ER_AGP',
 'Cells_Correlation_Costes_ER_DNA',
 'Cells_Correlation_Costes_ER_Mito',
 'Cells_Correlation_Costes_ER_RNA',
 'Cells_Correlation_Costes_Mito_AGP',
 'Cells_Correlation_Costes_Mito_DNA',
 'Cells_Correlation_Costes_Mito_ER',
 'Cells_Correlation_Costes_Mito_RNA',
 'Cells_Correlation_Costes_RNA_AGP',
 'Cells_Correlation_Costes_RNA_DNA',
 'Cells_Correlation_Costes_RNA_ER',
 'Cells_Correlation_Costes_RNA_Mito',
 'Cells_Correlation_K_AGP_DNA',
 'Cells_Correlation_K_AGP_ER',
 'Cells_Correlation_K_AGP_Mito',
 'Cells_Correlation_K_AGP_RNA',
 'Cells_Correlation_K_DNA_AGP',
 'Cells_Correlation_K_DNA_ER',
 'Cells_Correlation_K_DNA_Mito',
 'Cells_Correlation_K_DNA_RNA',
 'Cells_Correlation_K_ER_AGP',
 'Cells_Correlation_K_ER_DNA',
 'Cells_Correlation_K_ER_Mito',
 'Cells_Correlation_K_ER_RNA',
 'Cells_Correlation_K_Mito_AGP',
 'Cells_Correlation_K_Mito_DNA',
 'Cells_Correlation_K_Mito_ER',
 'Cells_Correlation_K_Mito_RNA',
 'Cells_Correlation_K_RNA_AGP',
 'Cells_Correlation_K_RNA_DNA',
 'Cells_Correlation_K_RNA_ER',
 'Cells_Correlation_K_RNA_Mito',
 'Cells_Correlation_Manders_AGP_DNA',
 'Cells_Correlation_Manders_AGP_ER',
 'Cells_Correlation_Manders_AGP_Mito',
 'Cells_Correlation_Manders_AGP_RNA',
 'Cells_Correlation_Manders_DNA_AGP',
 'Cells_Correlation_Manders_DNA_ER',
 'Cells_Correlation_Manders_DNA_Mito',
 'Cells_Correlation_Manders_DNA_RNA',
 'Cells_Correlation_Manders_ER_AGP',
 'Cells_Correlation_Manders_ER_DNA',
 'Cells_Correlation_Manders_ER_Mito',
 'Cells_Correlation_Manders_ER_RNA',
 'Cells_Correlation_Manders_Mito_AGP',
 'Cells_Correlation_Manders_Mito_DNA',
 'Cells_Correlation_Manders_Mito_ER',
 'Cells_Correlation_Manders_Mito_RNA',
 'Cells_Correlation_Manders_RNA_AGP',
 'Cells_Correlation_Manders_RNA_DNA',
 'Cells_Correlation_Manders_RNA_ER',
 'Cells_Correlation_Manders_RNA_Mito',
 'Cells_Correlation_Overlap_DNA_AGP',
 'Cells_Correlation_Overlap_DNA_ER',
 'Cells_Correlation_Overlap_DNA_Mito',
 'Cells_Correlation_Overlap_DNA_RNA',
 'Cells_Correlation_Overlap_ER_AGP',
 'Cells_Correlation_Overlap_ER_RNA',
 'Cells_Correlation_Overlap_Mito_AGP',
 'Cells_Correlation_Overlap_Mito_ER',
 'Cells_Correlation_Overlap_Mito_RNA',
 'Cells_Correlation_Overlap_RNA_AGP',
 'Cells_Correlation_RWC_AGP_DNA',
 'Cells_Correlation_RWC_AGP_ER',
 'Cells_Correlation_RWC_AGP_Mito',
 'Cells_Correlation_RWC_AGP_RNA',
 'Cells_Correlation_RWC_DNA_AGP',
 'Cells_Correlation_RWC_DNA_ER',
 'Cells_Correlation_RWC_DNA_Mito',
 'Cells_Correlation_RWC_DNA_RNA',
 'Cells_Correlation_RWC_ER_AGP',
 'Cells_Correlation_RWC_ER_DNA',
 'Cells_Correlation_RWC_ER_Mito',
 'Cells_Correlation_RWC_ER_RNA',
 'Cells_Correlation_RWC_Mito_AGP',
 'Cells_Correlation_RWC_Mito_DNA',
 'Cells_Correlation_RWC_Mito_ER',
 'Cells_Correlation_RWC_Mito_RNA',
 'Cells_Correlation_RWC_RNA_AGP',
 'Cells_Correlation_RWC_RNA_DNA',
 'Cells_Correlation_RWC_RNA_ER',
 'Cells_Correlation_RWC_RNA_Mito',
 'Cells_Granularity_10_AGP',
 'Cells_Granularity_10_ER',
 'Cells_Granularity_10_Mito',
 'Cells_Granularity_10_RNA',
 'Cells_Granularity_11_AGP',
 'Cells_Granularity_11_ER',
 'Cells_Granularity_11_Mito',
 'Cells_Granularity_11_RNA',
 'Cells_Granularity_12_AGP',
 'Cells_Granularity_12_ER',
 'Cells_Granularity_12_Mito',
 'Cells_Granularity_12_RNA',
 'Cells_Granularity_13_AGP',
 'Cells_Granularity_13_ER',
 'Cells_Granularity_13_Mito',
 'Cells_Granularity_13_RNA',
 'Cells_Granularity_14_AGP',
 'Cells_Granularity_14_ER',
 'Cells_Granularity_14_Mito',
 'Cells_Granularity_14_RNA',
 'Cells_Granularity_15_AGP',
 'Cells_Granularity_15_ER',
 'Cells_Granularity_15_Mito',
 'Cells_Granularity_15_RNA',
 'Cells_Granularity_16_AGP',
 'Cells_Granularity_16_ER',
 'Cells_Granularity_16_Mito',
 'Cells_Granularity_16_RNA',
 'Cells_Granularity_1_AGP',
 'Cells_Granularity_1_ER',
 'Cells_Granularity_1_Mito',
 'Cells_Granularity_1_RNA',
 'Cells_Granularity_2_AGP',
 'Cells_Granularity_2_ER',
 'Cells_Granularity_2_Mito',
 'Cells_Granularity_2_RNA',
 'Cells_Granularity_3_AGP',
 'Cells_Granularity_3_ER',
 'Cells_Granularity_3_Mito',
 'Cells_Granularity_3_RNA',
 'Cells_Granularity_4_AGP',
 'Cells_Granularity_4_ER',
 'Cells_Granularity_4_Mito',
 'Cells_Granularity_4_RNA',
 'Cells_Granularity_5_AGP',
 'Cells_Granularity_5_ER',
 'Cells_Granularity_5_Mito',
 'Cells_Granularity_5_RNA',
 'Cells_Granularity_6_AGP',
 'Cells_Granularity_6_ER',
 'Cells_Granularity_6_Mito',
 'Cells_Granularity_6_RNA',
 'Cells_Granularity_7_AGP',
 'Cells_Granularity_7_ER',
 'Cells_Granularity_7_Mito',
 'Cells_Granularity_7_RNA',
 'Cells_Granularity_8_AGP',
 'Cells_Granularity_8_ER',
 'Cells_Granularity_8_Mito',
 'Cells_Granularity_8_RNA',
 'Cells_Granularity_9_AGP',
 'Cells_Granularity_9_ER',
 'Cells_Granularity_9_Mito',
 'Cells_Granularity_9_RNA',
 'Cells_Intensity_IntegratedIntensityEdge_AGP',
 'Cells_Intensity_IntegratedIntensityEdge_DNA',
 'Cells_Intensity_IntegratedIntensityEdge_ER',
 'Cells_Intensity_IntegratedIntensityEdge_Mito',
 'Cells_Intensity_IntegratedIntensityEdge_RNA',
 'Cells_Intensity_IntegratedIntensity_AGP',
 'Cells_Intensity_IntegratedIntensity_DNA',
 'Cells_Intensity_IntegratedIntensity_ER',
 'Cells_Intensity_IntegratedIntensity_Mito',
 'Cells_Intensity_IntegratedIntensity_RNA',
 'Cells_Intensity_LowerQuartileIntensity_AGP',
 'Cells_Intensity_LowerQuartileIntensity_DNA',
 'Cells_Intensity_LowerQuartileIntensity_ER',
 'Cells_Intensity_LowerQuartileIntensity_Mito',
 'Cells_Intensity_LowerQuartileIntensity_RNA',
 'Cells_Intensity_MADIntensity_AGP',
 'Cells_Intensity_MADIntensity_DNA',
 'Cells_Intensity_MADIntensity_ER',
 'Cells_Intensity_MADIntensity_Mito',
 'Cells_Intensity_MADIntensity_RNA',
 'Cells_Intensity_MassDisplacement_AGP',
 'Cells_Intensity_MassDisplacement_DNA',
 'Cells_Intensity_MassDisplacement_ER',
 'Cells_Intensity_MassDisplacement_Mito',
 'Cells_Intensity_MassDisplacement_RNA',
 'Cells_Intensity_MaxIntensityEdge_AGP',
 'Cells_Intensity_MaxIntensityEdge_DNA',
 'Cells_Intensity_MaxIntensityEdge_ER',
 'Cells_Intensity_MaxIntensityEdge_Mito',
 'Cells_Intensity_MaxIntensityEdge_RNA',
 'Cells_Intensity_MaxIntensity_AGP',
 'Cells_Intensity_MaxIntensity_DNA',
 'Cells_Intensity_MaxIntensity_ER',
 'Cells_Intensity_MaxIntensity_Mito',
 'Cells_Intensity_MaxIntensity_RNA',
 'Cells_Intensity_MeanIntensityEdge_AGP',
 'Cells_Intensity_MeanIntensityEdge_DNA',
 'Cells_Intensity_MeanIntensityEdge_ER',
 'Cells_Intensity_MeanIntensityEdge_Mito',
 'Cells_Intensity_MeanIntensityEdge_RNA',
 'Cells_Intensity_MeanIntensity_AGP',
 'Cells_Intensity_MeanIntensity_DNA',
 'Cells_Intensity_MeanIntensity_ER',
 'Cells_Intensity_MeanIntensity_Mito',
 'Cells_Intensity_MeanIntensity_RNA',
 'Cells_Intensity_MedianIntensity_AGP',
 'Cells_Intensity_MedianIntensity_DNA',
 'Cells_Intensity_MedianIntensity_ER',
 'Cells_Intensity_MedianIntensity_Mito',
 'Cells_Intensity_MedianIntensity_RNA',
 'Cells_Intensity_MinIntensityEdge_AGP',
 'Cells_Intensity_MinIntensityEdge_DNA',
 'Cells_Intensity_MinIntensityEdge_ER',
 'Cells_Intensity_MinIntensityEdge_Mito',
 'Cells_Intensity_MinIntensityEdge_RNA',
 'Cells_Intensity_MinIntensity_AGP',
 'Cells_Intensity_MinIntensity_DNA',
 'Cells_Intensity_MinIntensity_ER',
 'Cells_Intensity_MinIntensity_Mito',
 'Cells_Intensity_MinIntensity_RNA',
 'Cells_Intensity_StdIntensityEdge_AGP',
 'Cells_Intensity_StdIntensityEdge_DNA',
 'Cells_Intensity_StdIntensityEdge_ER',
 'Cells_Intensity_StdIntensityEdge_Mito',
 'Cells_Intensity_StdIntensityEdge_RNA',
 'Cells_Intensity_StdIntensity_AGP',
 'Cells_Intensity_StdIntensity_DNA',
 'Cells_Intensity_StdIntensity_ER',
 'Cells_Intensity_StdIntensity_Mito',
 'Cells_Intensity_StdIntensity_RNA',
 'Cells_Intensity_UpperQuartileIntensity_AGP',
 'Cells_Intensity_UpperQuartileIntensity_DNA',
 'Cells_Intensity_UpperQuartileIntensity_ER',
 'Cells_Intensity_UpperQuartileIntensity_Mito',
 'Cells_Intensity_UpperQuartileIntensity_RNA',
 'Cells_Location_CenterMassIntensity_X_AGP',
 'Cells_Location_CenterMassIntensity_X_DNA',
 'Cells_Location_CenterMassIntensity_X_ER',
 'Cells_Location_CenterMassIntensity_X_Mito',
 'Cells_Location_CenterMassIntensity_X_RNA',
 'Cells_Location_CenterMassIntensity_Y_AGP',
 'Cells_Location_CenterMassIntensity_Y_DNA',
 'Cells_Location_CenterMassIntensity_Y_ER',
 'Cells_Location_CenterMassIntensity_Y_Mito',
 'Cells_Location_CenterMassIntensity_Y_RNA',
 'Cells_Location_Center_X',
 'Cells_Location_Center_Y',
 'Cells_Location_MaxIntensity_X_AGP',
 'Cells_Location_MaxIntensity_X_DNA',
 'Cells_Location_MaxIntensity_X_ER',
 'Cells_Location_MaxIntensity_X_Mito',
 'Cells_Location_MaxIntensity_X_RNA',
 'Cells_Location_MaxIntensity_Y_AGP',
 'Cells_Location_MaxIntensity_Y_DNA',
 'Cells_Location_MaxIntensity_Y_ER',
 'Cells_Location_MaxIntensity_Y_Mito',
 'Cells_Location_MaxIntensity_Y_RNA',
 'Cells_Neighbors_AngleBetweenNeighbors_5',
 'Cells_Neighbors_AngleBetweenNeighbors_Adjacent',
 'Cells_Neighbors_FirstClosestDistance_5',
 'Cells_Neighbors_FirstClosestDistance_Adjacent',
 'Cells_Neighbors_FirstClosestObjectNumber_5',
 'Cells_Neighbors_FirstClosestObjectNumber_Adjacent',
 'Cells_Neighbors_NumberOfNeighbors_5',
 'Cells_Neighbors_NumberOfNeighbors_Adjacent',
 'Cells_Neighbors_PercentTouching_5',
 'Cells_Neighbors_PercentTouching_Adjacent',
 'Cells_Neighbors_SecondClosestDistance_5',
 'Cells_Neighbors_SecondClosestDistance_Adjacent',
 'Cells_Neighbors_SecondClosestObjectNumber_5',
 'Cells_Neighbors_SecondClosestObjectNumber_Adjacent',
 'Cells_Number_Object_Number',
 'Cells_Parent_Nuclei',
 'Cells_RadialDistribution_FracAtD_AGP_1of4',
 'Cells_RadialDistribution_FracAtD_AGP_2of4',
 'Cells_RadialDistribution_FracAtD_AGP_3of4',
 'Cells_RadialDistribution_FracAtD_AGP_4of4',
 'Cells_RadialDistribution_FracAtD_DNA_1of4',
 'Cells_RadialDistribution_FracAtD_DNA_2of4',
 'Cells_RadialDistribution_FracAtD_DNA_3of4',
 'Cells_RadialDistribution_FracAtD_DNA_4of4',
 'Cells_RadialDistribution_FracAtD_ER_1of4',
 'Cells_RadialDistribution_FracAtD_ER_2of4',
 'Cells_RadialDistribution_FracAtD_ER_3of4',
 'Cells_RadialDistribution_FracAtD_ER_4of4',
 'Cells_RadialDistribution_FracAtD_Mito_1of4',
 'Cells_RadialDistribution_FracAtD_Mito_2of4',
 'Cells_RadialDistribution_FracAtD_Mito_3of4',
 'Cells_RadialDistribution_FracAtD_Mito_4of4',
 'Cells_RadialDistribution_FracAtD_RNA_1of4',
 'Cells_RadialDistribution_FracAtD_RNA_2of4',
 'Cells_RadialDistribution_FracAtD_RNA_3of4',
 'Cells_RadialDistribution_FracAtD_RNA_4of4',
 'Cells_RadialDistribution_MeanFrac_AGP_1of4',
 'Cells_RadialDistribution_MeanFrac_AGP_2of4',
 'Cells_RadialDistribution_MeanFrac_AGP_3of4',
 'Cells_RadialDistribution_MeanFrac_AGP_4of4',
 'Cells_RadialDistribution_MeanFrac_DNA_1of4',
 'Cells_RadialDistribution_MeanFrac_DNA_2of4',
 'Cells_RadialDistribution_MeanFrac_DNA_3of4',
 'Cells_RadialDistribution_MeanFrac_DNA_4of4',
 'Cells_RadialDistribution_MeanFrac_ER_1of4',
 'Cells_RadialDistribution_MeanFrac_ER_2of4',
 'Cells_RadialDistribution_MeanFrac_ER_3of4',
 'Cells_RadialDistribution_MeanFrac_ER_4of4',
 'Cells_RadialDistribution_MeanFrac_Mito_1of4',
 'Cells_RadialDistribution_MeanFrac_Mito_2of4',
 'Cells_RadialDistribution_MeanFrac_Mito_3of4',
 'Cells_RadialDistribution_MeanFrac_Mito_4of4',
 'Cells_RadialDistribution_MeanFrac_RNA_1of4',
 'Cells_RadialDistribution_MeanFrac_RNA_2of4',
 'Cells_RadialDistribution_MeanFrac_RNA_3of4',
 'Cells_RadialDistribution_MeanFrac_RNA_4of4',
 'Cells_RadialDistribution_RadialCV_AGP_1of4',
 'Cells_RadialDistribution_RadialCV_AGP_2of4',
 'Cells_RadialDistribution_RadialCV_AGP_3of4',
 'Cells_RadialDistribution_RadialCV_AGP_4of4',
 'Cells_RadialDistribution_RadialCV_DNA_1of4',
 'Cells_RadialDistribution_RadialCV_DNA_2of4',
 'Cells_RadialDistribution_RadialCV_DNA_3of4',
 'Cells_RadialDistribution_RadialCV_DNA_4of4',
 'Cells_RadialDistribution_RadialCV_ER_1of4',
 'Cells_RadialDistribution_RadialCV_ER_2of4',
 'Cells_RadialDistribution_RadialCV_ER_3of4',
 'Cells_RadialDistribution_RadialCV_ER_4of4',
 'Cells_RadialDistribution_RadialCV_Mito_1of4',
 'Cells_RadialDistribution_RadialCV_Mito_2of4',
 'Cells_RadialDistribution_RadialCV_Mito_3of4',
 'Cells_RadialDistribution_RadialCV_Mito_4of4',
 'Cells_RadialDistribution_RadialCV_RNA_1of4',
 'Cells_RadialDistribution_RadialCV_RNA_2of4',
 'Cells_RadialDistribution_RadialCV_RNA_3of4',
 'Cells_RadialDistribution_RadialCV_RNA_4of4',
 'Cells_Texture_AngularSecondMoment_AGP_10_0',
 'Cells_Texture_AngularSecondMoment_AGP_3_0',
 'Cells_Texture_AngularSecondMoment_AGP_5_0',
 'Cells_Texture_AngularSecondMoment_DNA_10_0',
 'Cells_Texture_AngularSecondMoment_DNA_3_0',
 'Cells_Texture_AngularSecondMoment_DNA_5_0',
 'Cells_Texture_AngularSecondMoment_ER_10_0',
 'Cells_Texture_AngularSecondMoment_ER_3_0',
 'Cells_Texture_AngularSecondMoment_ER_5_0',
 'Cells_Texture_AngularSecondMoment_Mito_10_0',
 'Cells_Texture_AngularSecondMoment_Mito_3_0',
 'Cells_Texture_AngularSecondMoment_Mito_5_0',
 'Cells_Texture_AngularSecondMoment_RNA_10_0',
 'Cells_Texture_AngularSecondMoment_RNA_3_0',
 'Cells_Texture_AngularSecondMoment_RNA_5_0',
 'Cells_Texture_Contrast_AGP_10_0',
 'Cells_Texture_Contrast_AGP_3_0',
 'Cells_Texture_Contrast_AGP_5_0',
 'Cells_Texture_Contrast_DNA_10_0',
 'Cells_Texture_Contrast_DNA_3_0',
 'Cells_Texture_Contrast_DNA_5_0',
 'Cells_Texture_Contrast_ER_10_0',
 'Cells_Texture_Contrast_ER_3_0',
 'Cells_Texture_Contrast_ER_5_0',
 'Cells_Texture_Contrast_Mito_10_0',
 'Cells_Texture_Contrast_Mito_3_0',
 'Cells_Texture_Contrast_Mito_5_0',
 'Cells_Texture_Contrast_RNA_10_0',
 'Cells_Texture_Contrast_RNA_3_0',
 'Cells_Texture_Contrast_RNA_5_0',
 'Cells_Texture_Correlation_AGP_10_0',
 'Cells_Texture_Correlation_AGP_3_0',
 'Cells_Texture_Correlation_AGP_5_0',
 'Cells_Texture_Correlation_DNA_10_0',
 'Cells_Texture_Correlation_DNA_3_0',
 'Cells_Texture_Correlation_DNA_5_0',
 'Cells_Texture_Correlation_ER_10_0',
 'Cells_Texture_Correlation_ER_3_0',
 'Cells_Texture_Correlation_ER_5_0',
 'Cells_Texture_Correlation_Mito_10_0',
 'Cells_Texture_Correlation_Mito_3_0',
 'Cells_Texture_Correlation_Mito_5_0',
 'Cells_Texture_Correlation_RNA_10_0',
 'Cells_Texture_Correlation_RNA_3_0',
 'Cells_Texture_Correlation_RNA_5_0',
 'Cells_Texture_DifferenceEntropy_AGP_10_0',
 'Cells_Texture_DifferenceEntropy_AGP_3_0',
 'Cells_Texture_DifferenceEntropy_AGP_5_0',
 'Cells_Texture_DifferenceEntropy_DNA_10_0',
 'Cells_Texture_DifferenceEntropy_DNA_3_0',
 'Cells_Texture_DifferenceEntropy_DNA_5_0',
 'Cells_Texture_DifferenceEntropy_ER_10_0',
 'Cells_Texture_DifferenceEntropy_ER_3_0',
 'Cells_Texture_DifferenceEntropy_ER_5_0',
 'Cells_Texture_DifferenceEntropy_Mito_10_0',
 'Cells_Texture_DifferenceEntropy_Mito_3_0',
 'Cells_Texture_DifferenceEntropy_Mito_5_0',
 'Cells_Texture_DifferenceEntropy_RNA_10_0',
 'Cells_Texture_DifferenceEntropy_RNA_3_0',
 'Cells_Texture_DifferenceEntropy_RNA_5_0',
 'Cells_Texture_DifferenceVariance_AGP_10_0',
 'Cells_Texture_DifferenceVariance_AGP_3_0',
 'Cells_Texture_DifferenceVariance_AGP_5_0',
 'Cells_Texture_DifferenceVariance_DNA_10_0',
 'Cells_Texture_DifferenceVariance_DNA_3_0',
 'Cells_Texture_DifferenceVariance_DNA_5_0',
 'Cells_Texture_DifferenceVariance_ER_10_0',
 'Cells_Texture_DifferenceVariance_ER_3_0',
 'Cells_Texture_DifferenceVariance_ER_5_0',
 'Cells_Texture_DifferenceVariance_Mito_10_0',
 'Cells_Texture_DifferenceVariance_Mito_3_0',
 'Cells_Texture_DifferenceVariance_Mito_5_0',
 'Cells_Texture_DifferenceVariance_RNA_10_0',
 'Cells_Texture_DifferenceVariance_RNA_3_0',
 'Cells_Texture_DifferenceVariance_RNA_5_0',
 'Cells_Texture_Entropy_AGP_10_0',
 'Cells_Texture_Entropy_AGP_3_0',
 'Cells_Texture_Entropy_AGP_5_0',
 'Cells_Texture_Entropy_DNA_10_0',
 'Cells_Texture_Entropy_DNA_3_0',
 'Cells_Texture_Entropy_DNA_5_0',
 'Cells_Texture_Entropy_ER_10_0',
 'Cells_Texture_Entropy_ER_3_0',
 'Cells_Texture_Entropy_ER_5_0',
 'Cells_Texture_Entropy_Mito_10_0',
 'Cells_Texture_Entropy_Mito_3_0',
 'Cells_Texture_Entropy_Mito_5_0',
 'Cells_Texture_Entropy_RNA_10_0',
 'Cells_Texture_Entropy_RNA_3_0',
 'Cells_Texture_Entropy_RNA_5_0',
 'Cells_Texture_Gabor_AGP_10',
 'Cells_Texture_Gabor_AGP_3',
 'Cells_Texture_Gabor_AGP_5',
 'Cells_Texture_Gabor_DNA_10',
 'Cells_Texture_Gabor_DNA_3',
 'Cells_Texture_Gabor_DNA_5',
 'Cells_Texture_Gabor_ER_10',
 'Cells_Texture_Gabor_ER_3',
 'Cells_Texture_Gabor_ER_5',
 'Cells_Texture_Gabor_Mito_10',
 'Cells_Texture_Gabor_Mito_3',
 'Cells_Texture_Gabor_Mito_5',
 'Cells_Texture_Gabor_RNA_10',
 'Cells_Texture_Gabor_RNA_3',
 'Cells_Texture_Gabor_RNA_5',
 'Cells_Texture_InfoMeas1_AGP_10_0',
 'Cells_Texture_InfoMeas1_AGP_3_0',
 'Cells_Texture_InfoMeas1_AGP_5_0',
 'Cells_Texture_InfoMeas1_DNA_10_0',
 'Cells_Texture_InfoMeas1_DNA_3_0',
 'Cells_Texture_InfoMeas1_DNA_5_0',
 'Cells_Texture_InfoMeas1_ER_10_0',
 'Cells_Texture_InfoMeas1_ER_3_0',
 'Cells_Texture_InfoMeas1_ER_5_0',
 'Cells_Texture_InfoMeas1_Mito_10_0',
 'Cells_Texture_InfoMeas1_Mito_3_0',
 'Cells_Texture_InfoMeas1_Mito_5_0',
 'Cells_Texture_InfoMeas1_RNA_10_0',
 'Cells_Texture_InfoMeas1_RNA_3_0',
 'Cells_Texture_InfoMeas1_RNA_5_0',
 'Cells_Texture_InfoMeas2_AGP_10_0',
 'Cells_Texture_InfoMeas2_AGP_3_0',
 'Cells_Texture_InfoMeas2_AGP_5_0',
 'Cells_Texture_InfoMeas2_DNA_10_0',
 'Cells_Texture_InfoMeas2_DNA_3_0',
 'Cells_Texture_InfoMeas2_DNA_5_0',
 'Cells_Texture_InfoMeas2_ER_10_0',
 'Cells_Texture_InfoMeas2_ER_3_0',
 'Cells_Texture_InfoMeas2_ER_5_0',
 'Cells_Texture_InfoMeas2_Mito_10_0',
 'Cells_Texture_InfoMeas2_Mito_3_0',
 'Cells_Texture_InfoMeas2_Mito_5_0',
 'Cells_Texture_InfoMeas2_RNA_10_0',
 'Cells_Texture_InfoMeas2_RNA_3_0',
 'Cells_Texture_InfoMeas2_RNA_5_0',
 'Cells_Texture_InverseDifferenceMoment_AGP_10_0',
 'Cells_Texture_InverseDifferenceMoment_AGP_3_0',
 'Cells_Texture_InverseDifferenceMoment_AGP_5_0',
 'Cells_Texture_InverseDifferenceMoment_DNA_10_0',
 'Cells_Texture_InverseDifferenceMoment_DNA_3_0',
 'Cells_Texture_InverseDifferenceMoment_DNA_5_0',
 'Cells_Texture_InverseDifferenceMoment_ER_10_0',
 'Cells_Texture_InverseDifferenceMoment_ER_3_0',
 'Cells_Texture_InverseDifferenceMoment_ER_5_0',
 'Cells_Texture_InverseDifferenceMoment_Mito_10_0',
 'Cells_Texture_InverseDifferenceMoment_Mito_3_0',
 'Cells_Texture_InverseDifferenceMoment_Mito_5_0',
 'Cells_Texture_InverseDifferenceMoment_RNA_10_0',
 'Cells_Texture_InverseDifferenceMoment_RNA_3_0',
 'Cells_Texture_InverseDifferenceMoment_RNA_5_0',
 'Cells_Texture_SumAverage_AGP_10_0',
 'Cells_Texture_SumAverage_AGP_3_0',
 'Cells_Texture_SumAverage_AGP_5_0',
 'Cells_Texture_SumAverage_DNA_10_0',
 'Cells_Texture_SumAverage_DNA_3_0',
 'Cells_Texture_SumAverage_DNA_5_0',
 'Cells_Texture_SumAverage_ER_10_0',
 'Cells_Texture_SumAverage_ER_3_0',
 'Cells_Texture_SumAverage_ER_5_0',
 'Cells_Texture_SumAverage_Mito_10_0',
 'Cells_Texture_SumAverage_Mito_3_0',
 'Cells_Texture_SumAverage_Mito_5_0',
 'Cells_Texture_SumAverage_RNA_10_0',
 'Cells_Texture_SumAverage_RNA_3_0',
 'Cells_Texture_SumAverage_RNA_5_0',
 'Cells_Texture_SumEntropy_AGP_10_0',
 'Cells_Texture_SumEntropy_AGP_3_0',
 'Cells_Texture_SumEntropy_AGP_5_0',
 'Cells_Texture_SumEntropy_DNA_10_0',
 'Cells_Texture_SumEntropy_DNA_3_0',
 'Cells_Texture_SumEntropy_DNA_5_0',
 'Cells_Texture_SumEntropy_ER_10_0',
 'Cells_Texture_SumEntropy_ER_3_0',
 'Cells_Texture_SumEntropy_ER_5_0',
 'Cells_Texture_SumEntropy_Mito_10_0',
 'Cells_Texture_SumEntropy_Mito_3_0',
 'Cells_Texture_SumEntropy_Mito_5_0',
 'Cells_Texture_SumEntropy_RNA_10_0',
 'Cells_Texture_SumEntropy_RNA_3_0',
 'Cells_Texture_SumEntropy_RNA_5_0',
 'Cells_Texture_SumVariance_AGP_10_0',
 'Cells_Texture_SumVariance_AGP_3_0',
 'Cells_Texture_SumVariance_AGP_5_0',
 'Cells_Texture_SumVariance_DNA_10_0',
 'Cells_Texture_SumVariance_DNA_3_0',
 'Cells_Texture_SumVariance_DNA_5_0',
 'Cells_Texture_SumVariance_ER_10_0',
 'Cells_Texture_SumVariance_ER_3_0',
 'Cells_Texture_SumVariance_ER_5_0',
 'Cells_Texture_SumVariance_Mito_10_0',
 'Cells_Texture_SumVariance_Mito_3_0',
 'Cells_Texture_SumVariance_Mito_5_0',
 'Cells_Texture_SumVariance_RNA_10_0',
 'Cells_Texture_SumVariance_RNA_3_0',
 'Cells_Texture_SumVariance_RNA_5_0',
 'Cells_Texture_Variance_AGP_10_0',
 'Cells_Texture_Variance_AGP_3_0',
 'Cells_Texture_Variance_AGP_5_0',
 'Cells_Texture_Variance_DNA_10_0',
 'Cells_Texture_Variance_DNA_3_0',
 'Cells_Texture_Variance_DNA_5_0',
 'Cells_Texture_Variance_ER_10_0',
 'Cells_Texture_Variance_ER_3_0',
 'Cells_Texture_Variance_ER_5_0',
 'Cells_Texture_Variance_Mito_10_0',
 'Cells_Texture_Variance_Mito_3_0',
 'Cells_Texture_Variance_Mito_5_0',
 'Cells_Texture_Variance_RNA_10_0',
 'Cells_Texture_Variance_RNA_3_0',
 'Cells_Texture_Variance_RNA_5_0'
 </p>
</details>

<details><summary>`Cytoplasm` Table</summary>
<p>
'TableNumber',
 'ImageNumber',
 'ObjectNumber',
 'Cytoplasm_AreaShape_Area',
 'Cytoplasm_AreaShape_Center_X',
 'Cytoplasm_AreaShape_Center_Y',
 'Cytoplasm_AreaShape_Compactness',
 'Cytoplasm_AreaShape_Eccentricity',
 'Cytoplasm_AreaShape_EulerNumber',
 'Cytoplasm_AreaShape_Extent',
 'Cytoplasm_AreaShape_FormFactor',
 'Cytoplasm_AreaShape_MajorAxisLength',
 'Cytoplasm_AreaShape_MaxFeretDiameter',
 'Cytoplasm_AreaShape_MaximumRadius',
 'Cytoplasm_AreaShape_MeanRadius',
 'Cytoplasm_AreaShape_MedianRadius',
 'Cytoplasm_AreaShape_MinFeretDiameter',
 'Cytoplasm_AreaShape_MinorAxisLength',
 'Cytoplasm_AreaShape_Orientation',
 'Cytoplasm_AreaShape_Perimeter',
 'Cytoplasm_AreaShape_Solidity',
 'Cytoplasm_AreaShape_Zernike_0_0',
 'Cytoplasm_AreaShape_Zernike_1_1',
 'Cytoplasm_AreaShape_Zernike_2_0',
 'Cytoplasm_AreaShape_Zernike_2_2',
 'Cytoplasm_AreaShape_Zernike_3_1',
 'Cytoplasm_AreaShape_Zernike_3_3',
 'Cytoplasm_AreaShape_Zernike_4_0',
 'Cytoplasm_AreaShape_Zernike_4_2',
 'Cytoplasm_AreaShape_Zernike_4_4',
 'Cytoplasm_AreaShape_Zernike_5_1',
 'Cytoplasm_AreaShape_Zernike_5_3',
 'Cytoplasm_AreaShape_Zernike_5_5',
 'Cytoplasm_AreaShape_Zernike_6_0',
 'Cytoplasm_AreaShape_Zernike_6_2',
 'Cytoplasm_AreaShape_Zernike_6_4',
 'Cytoplasm_AreaShape_Zernike_6_6',
 'Cytoplasm_AreaShape_Zernike_7_1',
 'Cytoplasm_AreaShape_Zernike_7_3',
 'Cytoplasm_AreaShape_Zernike_7_5',
 'Cytoplasm_AreaShape_Zernike_7_7',
 'Cytoplasm_AreaShape_Zernike_8_0',
 'Cytoplasm_AreaShape_Zernike_8_2',
 'Cytoplasm_AreaShape_Zernike_8_4',
 'Cytoplasm_AreaShape_Zernike_8_6',
 'Cytoplasm_AreaShape_Zernike_8_8',
 'Cytoplasm_AreaShape_Zernike_9_1',
 'Cytoplasm_AreaShape_Zernike_9_3',
 'Cytoplasm_AreaShape_Zernike_9_5',
 'Cytoplasm_AreaShape_Zernike_9_7',
 'Cytoplasm_AreaShape_Zernike_9_9',
 'Cytoplasm_Correlation_Correlation_DNA_AGP',
 'Cytoplasm_Correlation_Correlation_DNA_ER',
 'Cytoplasm_Correlation_Correlation_DNA_Mito',
 'Cytoplasm_Correlation_Correlation_DNA_RNA',
 'Cytoplasm_Correlation_Correlation_ER_AGP',
 'Cytoplasm_Correlation_Correlation_ER_RNA',
 'Cytoplasm_Correlation_Correlation_Mito_AGP',
 'Cytoplasm_Correlation_Correlation_Mito_ER',
 'Cytoplasm_Correlation_Correlation_Mito_RNA',
 'Cytoplasm_Correlation_Correlation_RNA_AGP',
 'Cytoplasm_Correlation_Costes_AGP_DNA',
 'Cytoplasm_Correlation_Costes_AGP_ER',
 'Cytoplasm_Correlation_Costes_AGP_Mito',
 'Cytoplasm_Correlation_Costes_AGP_RNA',
 'Cytoplasm_Correlation_Costes_DNA_AGP',
 'Cytoplasm_Correlation_Costes_DNA_ER',
 'Cytoplasm_Correlation_Costes_DNA_Mito',
 'Cytoplasm_Correlation_Costes_DNA_RNA',
 'Cytoplasm_Correlation_Costes_ER_AGP',
 'Cytoplasm_Correlation_Costes_ER_DNA',
 'Cytoplasm_Correlation_Costes_ER_Mito',
 'Cytoplasm_Correlation_Costes_ER_RNA',
 'Cytoplasm_Correlation_Costes_Mito_AGP',
 'Cytoplasm_Correlation_Costes_Mito_DNA',
 'Cytoplasm_Correlation_Costes_Mito_ER',
 'Cytoplasm_Correlation_Costes_Mito_RNA',
 'Cytoplasm_Correlation_Costes_RNA_AGP',
 'Cytoplasm_Correlation_Costes_RNA_DNA',
 'Cytoplasm_Correlation_Costes_RNA_ER',
 'Cytoplasm_Correlation_Costes_RNA_Mito',
 'Cytoplasm_Correlation_K_AGP_DNA',
 'Cytoplasm_Correlation_K_AGP_ER',
 'Cytoplasm_Correlation_K_AGP_Mito',
 'Cytoplasm_Correlation_K_AGP_RNA',
 'Cytoplasm_Correlation_K_DNA_AGP',
 'Cytoplasm_Correlation_K_DNA_ER',
 'Cytoplasm_Correlation_K_DNA_Mito',
 'Cytoplasm_Correlation_K_DNA_RNA',
 'Cytoplasm_Correlation_K_ER_AGP',
 'Cytoplasm_Correlation_K_ER_DNA',
 'Cytoplasm_Correlation_K_ER_Mito',
 'Cytoplasm_Correlation_K_ER_RNA',
 'Cytoplasm_Correlation_K_Mito_AGP',
 'Cytoplasm_Correlation_K_Mito_DNA',
 'Cytoplasm_Correlation_K_Mito_ER',
 'Cytoplasm_Correlation_K_Mito_RNA',
 'Cytoplasm_Correlation_K_RNA_AGP',
 'Cytoplasm_Correlation_K_RNA_DNA',
 'Cytoplasm_Correlation_K_RNA_ER',
 'Cytoplasm_Correlation_K_RNA_Mito',
 'Cytoplasm_Correlation_Manders_AGP_DNA',
 'Cytoplasm_Correlation_Manders_AGP_ER',
 'Cytoplasm_Correlation_Manders_AGP_Mito',
 'Cytoplasm_Correlation_Manders_AGP_RNA',
 'Cytoplasm_Correlation_Manders_DNA_AGP',
 'Cytoplasm_Correlation_Manders_DNA_ER',
 'Cytoplasm_Correlation_Manders_DNA_Mito',
 'Cytoplasm_Correlation_Manders_DNA_RNA',
 'Cytoplasm_Correlation_Manders_ER_AGP',
 'Cytoplasm_Correlation_Manders_ER_DNA',
 'Cytoplasm_Correlation_Manders_ER_Mito',
 'Cytoplasm_Correlation_Manders_ER_RNA',
 'Cytoplasm_Correlation_Manders_Mito_AGP',
 'Cytoplasm_Correlation_Manders_Mito_DNA',
 'Cytoplasm_Correlation_Manders_Mito_ER',
 'Cytoplasm_Correlation_Manders_Mito_RNA',
 'Cytoplasm_Correlation_Manders_RNA_AGP',
 'Cytoplasm_Correlation_Manders_RNA_DNA',
 'Cytoplasm_Correlation_Manders_RNA_ER',
 'Cytoplasm_Correlation_Manders_RNA_Mito',
 'Cytoplasm_Correlation_Overlap_DNA_AGP',
 'Cytoplasm_Correlation_Overlap_DNA_ER',
 'Cytoplasm_Correlation_Overlap_DNA_Mito',
 'Cytoplasm_Correlation_Overlap_DNA_RNA',
 'Cytoplasm_Correlation_Overlap_ER_AGP',
 'Cytoplasm_Correlation_Overlap_ER_RNA',
 'Cytoplasm_Correlation_Overlap_Mito_AGP',
 'Cytoplasm_Correlation_Overlap_Mito_ER',
 'Cytoplasm_Correlation_Overlap_Mito_RNA',
 'Cytoplasm_Correlation_Overlap_RNA_AGP',
 'Cytoplasm_Correlation_RWC_AGP_DNA',
 'Cytoplasm_Correlation_RWC_AGP_ER',
 'Cytoplasm_Correlation_RWC_AGP_Mito',
 'Cytoplasm_Correlation_RWC_AGP_RNA',
 'Cytoplasm_Correlation_RWC_DNA_AGP',
 'Cytoplasm_Correlation_RWC_DNA_ER',
 'Cytoplasm_Correlation_RWC_DNA_Mito',
 'Cytoplasm_Correlation_RWC_DNA_RNA',
 'Cytoplasm_Correlation_RWC_ER_AGP',
 'Cytoplasm_Correlation_RWC_ER_DNA',
 'Cytoplasm_Correlation_RWC_ER_Mito',
 'Cytoplasm_Correlation_RWC_ER_RNA',
 'Cytoplasm_Correlation_RWC_Mito_AGP',
 'Cytoplasm_Correlation_RWC_Mito_DNA',
 'Cytoplasm_Correlation_RWC_Mito_ER',
 'Cytoplasm_Correlation_RWC_Mito_RNA',
 'Cytoplasm_Correlation_RWC_RNA_AGP',
 'Cytoplasm_Correlation_RWC_RNA_DNA',
 'Cytoplasm_Correlation_RWC_RNA_ER',
 'Cytoplasm_Correlation_RWC_RNA_Mito',
 'Cytoplasm_Granularity_10_AGP',
 'Cytoplasm_Granularity_10_ER',
 'Cytoplasm_Granularity_10_Mito',
 'Cytoplasm_Granularity_10_RNA',
 'Cytoplasm_Granularity_11_AGP',
 'Cytoplasm_Granularity_11_ER',
 'Cytoplasm_Granularity_11_Mito',
 'Cytoplasm_Granularity_11_RNA',
 'Cytoplasm_Granularity_12_AGP',
 'Cytoplasm_Granularity_12_ER',
 'Cytoplasm_Granularity_12_Mito',
 'Cytoplasm_Granularity_12_RNA',
 'Cytoplasm_Granularity_13_AGP',
 'Cytoplasm_Granularity_13_ER',
 'Cytoplasm_Granularity_13_Mito',
 'Cytoplasm_Granularity_13_RNA',
 'Cytoplasm_Granularity_14_AGP',
 'Cytoplasm_Granularity_14_ER',
 'Cytoplasm_Granularity_14_Mito',
 'Cytoplasm_Granularity_14_RNA',
 'Cytoplasm_Granularity_15_AGP',
 'Cytoplasm_Granularity_15_ER',
 'Cytoplasm_Granularity_15_Mito',
 'Cytoplasm_Granularity_15_RNA',
 'Cytoplasm_Granularity_16_AGP',
 'Cytoplasm_Granularity_16_ER',
 'Cytoplasm_Granularity_16_Mito',
 'Cytoplasm_Granularity_16_RNA',
 'Cytoplasm_Granularity_1_AGP',
 'Cytoplasm_Granularity_1_ER',
 'Cytoplasm_Granularity_1_Mito',
 'Cytoplasm_Granularity_1_RNA',
 'Cytoplasm_Granularity_2_AGP',
 'Cytoplasm_Granularity_2_ER',
 'Cytoplasm_Granularity_2_Mito',
 'Cytoplasm_Granularity_2_RNA',
 'Cytoplasm_Granularity_3_AGP',
 'Cytoplasm_Granularity_3_ER',
 'Cytoplasm_Granularity_3_Mito',
 'Cytoplasm_Granularity_3_RNA',
 'Cytoplasm_Granularity_4_AGP',
 'Cytoplasm_Granularity_4_ER',
 'Cytoplasm_Granularity_4_Mito',
 'Cytoplasm_Granularity_4_RNA',
 'Cytoplasm_Granularity_5_AGP',
 'Cytoplasm_Granularity_5_ER',
 'Cytoplasm_Granularity_5_Mito',
 'Cytoplasm_Granularity_5_RNA',
 'Cytoplasm_Granularity_6_AGP',
 'Cytoplasm_Granularity_6_ER',
 'Cytoplasm_Granularity_6_Mito',
 'Cytoplasm_Granularity_6_RNA',
 'Cytoplasm_Granularity_7_AGP',
 'Cytoplasm_Granularity_7_ER',
 'Cytoplasm_Granularity_7_Mito',
 'Cytoplasm_Granularity_7_RNA',
 'Cytoplasm_Granularity_8_AGP',
 'Cytoplasm_Granularity_8_ER',
 'Cytoplasm_Granularity_8_Mito',
 'Cytoplasm_Granularity_8_RNA',
 'Cytoplasm_Granularity_9_AGP',
 'Cytoplasm_Granularity_9_ER',
 'Cytoplasm_Granularity_9_Mito',
 'Cytoplasm_Granularity_9_RNA',
 'Cytoplasm_Intensity_IntegratedIntensityEdge_AGP',
 'Cytoplasm_Intensity_IntegratedIntensityEdge_DNA',
 'Cytoplasm_Intensity_IntegratedIntensityEdge_ER',
 'Cytoplasm_Intensity_IntegratedIntensityEdge_Mito',
 'Cytoplasm_Intensity_IntegratedIntensityEdge_RNA',
 'Cytoplasm_Intensity_IntegratedIntensity_AGP',
 'Cytoplasm_Intensity_IntegratedIntensity_DNA',
 'Cytoplasm_Intensity_IntegratedIntensity_ER',
 'Cytoplasm_Intensity_IntegratedIntensity_Mito',
 'Cytoplasm_Intensity_IntegratedIntensity_RNA',
 'Cytoplasm_Intensity_LowerQuartileIntensity_AGP',
 'Cytoplasm_Intensity_LowerQuartileIntensity_DNA',
 'Cytoplasm_Intensity_LowerQuartileIntensity_ER',
 'Cytoplasm_Intensity_LowerQuartileIntensity_Mito',
 'Cytoplasm_Intensity_LowerQuartileIntensity_RNA',
 'Cytoplasm_Intensity_MADIntensity_AGP',
 'Cytoplasm_Intensity_MADIntensity_DNA',
 'Cytoplasm_Intensity_MADIntensity_ER',
 'Cytoplasm_Intensity_MADIntensity_Mito',
 'Cytoplasm_Intensity_MADIntensity_RNA',
 'Cytoplasm_Intensity_MassDisplacement_AGP',
 'Cytoplasm_Intensity_MassDisplacement_DNA',
 'Cytoplasm_Intensity_MassDisplacement_ER',
 'Cytoplasm_Intensity_MassDisplacement_Mito',
 'Cytoplasm_Intensity_MassDisplacement_RNA',
 'Cytoplasm_Intensity_MaxIntensityEdge_AGP',
 'Cytoplasm_Intensity_MaxIntensityEdge_DNA',
 'Cytoplasm_Intensity_MaxIntensityEdge_ER',
 'Cytoplasm_Intensity_MaxIntensityEdge_Mito',
 'Cytoplasm_Intensity_MaxIntensityEdge_RNA',
 'Cytoplasm_Intensity_MaxIntensity_AGP',
 'Cytoplasm_Intensity_MaxIntensity_DNA',
 'Cytoplasm_Intensity_MaxIntensity_ER',
 'Cytoplasm_Intensity_MaxIntensity_Mito',
 'Cytoplasm_Intensity_MaxIntensity_RNA',
 'Cytoplasm_Intensity_MeanIntensityEdge_AGP',
 'Cytoplasm_Intensity_MeanIntensityEdge_DNA',
 'Cytoplasm_Intensity_MeanIntensityEdge_ER',
 'Cytoplasm_Intensity_MeanIntensityEdge_Mito',
 'Cytoplasm_Intensity_MeanIntensityEdge_RNA',
 'Cytoplasm_Intensity_MeanIntensity_AGP',
 'Cytoplasm_Intensity_MeanIntensity_DNA',
 'Cytoplasm_Intensity_MeanIntensity_ER',
 'Cytoplasm_Intensity_MeanIntensity_Mito',
 'Cytoplasm_Intensity_MeanIntensity_RNA',
 'Cytoplasm_Intensity_MedianIntensity_AGP',
 'Cytoplasm_Intensity_MedianIntensity_DNA',
 'Cytoplasm_Intensity_MedianIntensity_ER',
 'Cytoplasm_Intensity_MedianIntensity_Mito',
 'Cytoplasm_Intensity_MedianIntensity_RNA',
 'Cytoplasm_Intensity_MinIntensityEdge_AGP',
 'Cytoplasm_Intensity_MinIntensityEdge_DNA',
 'Cytoplasm_Intensity_MinIntensityEdge_ER',
 'Cytoplasm_Intensity_MinIntensityEdge_Mito',
 'Cytoplasm_Intensity_MinIntensityEdge_RNA',
 'Cytoplasm_Intensity_MinIntensity_AGP',
 'Cytoplasm_Intensity_MinIntensity_DNA',
 'Cytoplasm_Intensity_MinIntensity_ER',
 'Cytoplasm_Intensity_MinIntensity_Mito',
 'Cytoplasm_Intensity_MinIntensity_RNA',
 'Cytoplasm_Intensity_StdIntensityEdge_AGP',
 'Cytoplasm_Intensity_StdIntensityEdge_DNA',
 'Cytoplasm_Intensity_StdIntensityEdge_ER',
 'Cytoplasm_Intensity_StdIntensityEdge_Mito',
 'Cytoplasm_Intensity_StdIntensityEdge_RNA',
 'Cytoplasm_Intensity_StdIntensity_AGP',
 'Cytoplasm_Intensity_StdIntensity_DNA',
 'Cytoplasm_Intensity_StdIntensity_ER',
 'Cytoplasm_Intensity_StdIntensity_Mito',
 'Cytoplasm_Intensity_StdIntensity_RNA',
 'Cytoplasm_Intensity_UpperQuartileIntensity_AGP',
 'Cytoplasm_Intensity_UpperQuartileIntensity_DNA',
 'Cytoplasm_Intensity_UpperQuartileIntensity_ER',
 'Cytoplasm_Intensity_UpperQuartileIntensity_Mito',
 'Cytoplasm_Intensity_UpperQuartileIntensity_RNA',
 'Cytoplasm_Location_CenterMassIntensity_X_AGP',
 'Cytoplasm_Location_CenterMassIntensity_X_DNA',
 'Cytoplasm_Location_CenterMassIntensity_X_ER',
 'Cytoplasm_Location_CenterMassIntensity_X_Mito',
 'Cytoplasm_Location_CenterMassIntensity_X_RNA',
 'Cytoplasm_Location_CenterMassIntensity_Y_AGP',
 'Cytoplasm_Location_CenterMassIntensity_Y_DNA',
 'Cytoplasm_Location_CenterMassIntensity_Y_ER',
 'Cytoplasm_Location_CenterMassIntensity_Y_Mito',
 'Cytoplasm_Location_CenterMassIntensity_Y_RNA',
 'Cytoplasm_Location_Center_X',
 'Cytoplasm_Location_Center_Y',
 'Cytoplasm_Location_MaxIntensity_X_AGP',
 'Cytoplasm_Location_MaxIntensity_X_DNA',
 'Cytoplasm_Location_MaxIntensity_X_ER',
 'Cytoplasm_Location_MaxIntensity_X_Mito',
 'Cytoplasm_Location_MaxIntensity_X_RNA',
 'Cytoplasm_Location_MaxIntensity_Y_AGP',
 'Cytoplasm_Location_MaxIntensity_Y_DNA',
 'Cytoplasm_Location_MaxIntensity_Y_ER',
 'Cytoplasm_Location_MaxIntensity_Y_Mito',
 'Cytoplasm_Location_MaxIntensity_Y_RNA',
 'Cytoplasm_Number_Object_Number',
 'Cytoplasm_Parent_Cells',
 'Cytoplasm_Parent_Nuclei',
 'Cytoplasm_RadialDistribution_FracAtD_AGP_1of4',
 'Cytoplasm_RadialDistribution_FracAtD_AGP_2of4',
 'Cytoplasm_RadialDistribution_FracAtD_AGP_3of4',
 'Cytoplasm_RadialDistribution_FracAtD_AGP_4of4',
 'Cytoplasm_RadialDistribution_FracAtD_DNA_1of4',
 'Cytoplasm_RadialDistribution_FracAtD_DNA_2of4',
 'Cytoplasm_RadialDistribution_FracAtD_DNA_3of4',
 'Cytoplasm_RadialDistribution_FracAtD_DNA_4of4',
 'Cytoplasm_RadialDistribution_FracAtD_ER_1of4',
 'Cytoplasm_RadialDistribution_FracAtD_ER_2of4',
 'Cytoplasm_RadialDistribution_FracAtD_ER_3of4',
 'Cytoplasm_RadialDistribution_FracAtD_ER_4of4',
 'Cytoplasm_RadialDistribution_FracAtD_Mito_1of4',
 'Cytoplasm_RadialDistribution_FracAtD_Mito_2of4',
 'Cytoplasm_RadialDistribution_FracAtD_Mito_3of4',
 'Cytoplasm_RadialDistribution_FracAtD_Mito_4of4',
 'Cytoplasm_RadialDistribution_FracAtD_RNA_1of4',
 'Cytoplasm_RadialDistribution_FracAtD_RNA_2of4',
 'Cytoplasm_RadialDistribution_FracAtD_RNA_3of4',
 'Cytoplasm_RadialDistribution_FracAtD_RNA_4of4',
 'Cytoplasm_RadialDistribution_MeanFrac_AGP_1of4',
 'Cytoplasm_RadialDistribution_MeanFrac_AGP_2of4',
 'Cytoplasm_RadialDistribution_MeanFrac_AGP_3of4',
 'Cytoplasm_RadialDistribution_MeanFrac_AGP_4of4',
 'Cytoplasm_RadialDistribution_MeanFrac_DNA_1of4',
 'Cytoplasm_RadialDistribution_MeanFrac_DNA_2of4',
 'Cytoplasm_RadialDistribution_MeanFrac_DNA_3of4',
 'Cytoplasm_RadialDistribution_MeanFrac_DNA_4of4',
 'Cytoplasm_RadialDistribution_MeanFrac_ER_1of4',
 'Cytoplasm_RadialDistribution_MeanFrac_ER_2of4',
 'Cytoplasm_RadialDistribution_MeanFrac_ER_3of4',
 'Cytoplasm_RadialDistribution_MeanFrac_ER_4of4',
 'Cytoplasm_RadialDistribution_MeanFrac_Mito_1of4',
 'Cytoplasm_RadialDistribution_MeanFrac_Mito_2of4',
 'Cytoplasm_RadialDistribution_MeanFrac_Mito_3of4',
 'Cytoplasm_RadialDistribution_MeanFrac_Mito_4of4',
 'Cytoplasm_RadialDistribution_MeanFrac_RNA_1of4',
 'Cytoplasm_RadialDistribution_MeanFrac_RNA_2of4',
 'Cytoplasm_RadialDistribution_MeanFrac_RNA_3of4',
 'Cytoplasm_RadialDistribution_MeanFrac_RNA_4of4',
 'Cytoplasm_RadialDistribution_RadialCV_AGP_1of4',
 'Cytoplasm_RadialDistribution_RadialCV_AGP_2of4',
 'Cytoplasm_RadialDistribution_RadialCV_AGP_3of4',
 'Cytoplasm_RadialDistribution_RadialCV_AGP_4of4',
 'Cytoplasm_RadialDistribution_RadialCV_DNA_1of4',
 'Cytoplasm_RadialDistribution_RadialCV_DNA_2of4',
 'Cytoplasm_RadialDistribution_RadialCV_DNA_3of4',
 'Cytoplasm_RadialDistribution_RadialCV_DNA_4of4',
 'Cytoplasm_RadialDistribution_RadialCV_ER_1of4',
 'Cytoplasm_RadialDistribution_RadialCV_ER_2of4',
 'Cytoplasm_RadialDistribution_RadialCV_ER_3of4',
 'Cytoplasm_RadialDistribution_RadialCV_ER_4of4',
 'Cytoplasm_RadialDistribution_RadialCV_Mito_1of4',
 'Cytoplasm_RadialDistribution_RadialCV_Mito_2of4',
 'Cytoplasm_RadialDistribution_RadialCV_Mito_3of4',
 'Cytoplasm_RadialDistribution_RadialCV_Mito_4of4',
 'Cytoplasm_RadialDistribution_RadialCV_RNA_1of4',
 'Cytoplasm_RadialDistribution_RadialCV_RNA_2of4',
 'Cytoplasm_RadialDistribution_RadialCV_RNA_3of4',
 'Cytoplasm_RadialDistribution_RadialCV_RNA_4of4',
 'Cytoplasm_Texture_AngularSecondMoment_AGP_10_0',
 'Cytoplasm_Texture_AngularSecondMoment_AGP_3_0',
 'Cytoplasm_Texture_AngularSecondMoment_AGP_5_0',
 'Cytoplasm_Texture_AngularSecondMoment_DNA_10_0',
 'Cytoplasm_Texture_AngularSecondMoment_DNA_3_0',
 'Cytoplasm_Texture_AngularSecondMoment_DNA_5_0',
 'Cytoplasm_Texture_AngularSecondMoment_ER_10_0',
 'Cytoplasm_Texture_AngularSecondMoment_ER_3_0',
 'Cytoplasm_Texture_AngularSecondMoment_ER_5_0',
 'Cytoplasm_Texture_AngularSecondMoment_Mito_10_0',
 'Cytoplasm_Texture_AngularSecondMoment_Mito_3_0',
 'Cytoplasm_Texture_AngularSecondMoment_Mito_5_0',
 'Cytoplasm_Texture_AngularSecondMoment_RNA_10_0',
 'Cytoplasm_Texture_AngularSecondMoment_RNA_3_0',
 'Cytoplasm_Texture_AngularSecondMoment_RNA_5_0',
 'Cytoplasm_Texture_Contrast_AGP_10_0',
 'Cytoplasm_Texture_Contrast_AGP_3_0',
 'Cytoplasm_Texture_Contrast_AGP_5_0',
 'Cytoplasm_Texture_Contrast_DNA_10_0',
 'Cytoplasm_Texture_Contrast_DNA_3_0',
 'Cytoplasm_Texture_Contrast_DNA_5_0',
 'Cytoplasm_Texture_Contrast_ER_10_0',
 'Cytoplasm_Texture_Contrast_ER_3_0',
 'Cytoplasm_Texture_Contrast_ER_5_0',
 'Cytoplasm_Texture_Contrast_Mito_10_0',
 'Cytoplasm_Texture_Contrast_Mito_3_0',
 'Cytoplasm_Texture_Contrast_Mito_5_0',
 'Cytoplasm_Texture_Contrast_RNA_10_0',
 'Cytoplasm_Texture_Contrast_RNA_3_0',
 'Cytoplasm_Texture_Contrast_RNA_5_0',
 'Cytoplasm_Texture_Correlation_AGP_10_0',
 'Cytoplasm_Texture_Correlation_AGP_3_0',
 'Cytoplasm_Texture_Correlation_AGP_5_0',
 'Cytoplasm_Texture_Correlation_DNA_10_0',
 'Cytoplasm_Texture_Correlation_DNA_3_0',
 'Cytoplasm_Texture_Correlation_DNA_5_0',
 'Cytoplasm_Texture_Correlation_ER_10_0',
 'Cytoplasm_Texture_Correlation_ER_3_0',
 'Cytoplasm_Texture_Correlation_ER_5_0',
 'Cytoplasm_Texture_Correlation_Mito_10_0',
 'Cytoplasm_Texture_Correlation_Mito_3_0',
 'Cytoplasm_Texture_Correlation_Mito_5_0',
 'Cytoplasm_Texture_Correlation_RNA_10_0',
 'Cytoplasm_Texture_Correlation_RNA_3_0',
 'Cytoplasm_Texture_Correlation_RNA_5_0',
 'Cytoplasm_Texture_DifferenceEntropy_AGP_10_0',
 'Cytoplasm_Texture_DifferenceEntropy_AGP_3_0',
 'Cytoplasm_Texture_DifferenceEntropy_AGP_5_0',
 'Cytoplasm_Texture_DifferenceEntropy_DNA_10_0',
 'Cytoplasm_Texture_DifferenceEntropy_DNA_3_0',
 'Cytoplasm_Texture_DifferenceEntropy_DNA_5_0',
 'Cytoplasm_Texture_DifferenceEntropy_ER_10_0',
 'Cytoplasm_Texture_DifferenceEntropy_ER_3_0',
 'Cytoplasm_Texture_DifferenceEntropy_ER_5_0',
 'Cytoplasm_Texture_DifferenceEntropy_Mito_10_0',
 'Cytoplasm_Texture_DifferenceEntropy_Mito_3_0',
 'Cytoplasm_Texture_DifferenceEntropy_Mito_5_0',
 'Cytoplasm_Texture_DifferenceEntropy_RNA_10_0',
 'Cytoplasm_Texture_DifferenceEntropy_RNA_3_0',
 'Cytoplasm_Texture_DifferenceEntropy_RNA_5_0',
 'Cytoplasm_Texture_DifferenceVariance_AGP_10_0',
 'Cytoplasm_Texture_DifferenceVariance_AGP_3_0',
 'Cytoplasm_Texture_DifferenceVariance_AGP_5_0',
 'Cytoplasm_Texture_DifferenceVariance_DNA_10_0',
 'Cytoplasm_Texture_DifferenceVariance_DNA_3_0',
 'Cytoplasm_Texture_DifferenceVariance_DNA_5_0',
 'Cytoplasm_Texture_DifferenceVariance_ER_10_0',
 'Cytoplasm_Texture_DifferenceVariance_ER_3_0',
 'Cytoplasm_Texture_DifferenceVariance_ER_5_0',
 'Cytoplasm_Texture_DifferenceVariance_Mito_10_0',
 'Cytoplasm_Texture_DifferenceVariance_Mito_3_0',
 'Cytoplasm_Texture_DifferenceVariance_Mito_5_0',
 'Cytoplasm_Texture_DifferenceVariance_RNA_10_0',
 'Cytoplasm_Texture_DifferenceVariance_RNA_3_0',
 'Cytoplasm_Texture_DifferenceVariance_RNA_5_0',
 'Cytoplasm_Texture_Entropy_AGP_10_0',
 'Cytoplasm_Texture_Entropy_AGP_3_0',
 'Cytoplasm_Texture_Entropy_AGP_5_0',
 'Cytoplasm_Texture_Entropy_DNA_10_0',
 'Cytoplasm_Texture_Entropy_DNA_3_0',
 'Cytoplasm_Texture_Entropy_DNA_5_0',
 'Cytoplasm_Texture_Entropy_ER_10_0',
 'Cytoplasm_Texture_Entropy_ER_3_0',
 'Cytoplasm_Texture_Entropy_ER_5_0',
 'Cytoplasm_Texture_Entropy_Mito_10_0',
 'Cytoplasm_Texture_Entropy_Mito_3_0',
 'Cytoplasm_Texture_Entropy_Mito_5_0',
 'Cytoplasm_Texture_Entropy_RNA_10_0',
 'Cytoplasm_Texture_Entropy_RNA_3_0',
 'Cytoplasm_Texture_Entropy_RNA_5_0',
 'Cytoplasm_Texture_Gabor_AGP_10',
 'Cytoplasm_Texture_Gabor_AGP_3',
 'Cytoplasm_Texture_Gabor_AGP_5',
 'Cytoplasm_Texture_Gabor_DNA_10',
 'Cytoplasm_Texture_Gabor_DNA_3',
 'Cytoplasm_Texture_Gabor_DNA_5',
 'Cytoplasm_Texture_Gabor_ER_10',
 'Cytoplasm_Texture_Gabor_ER_3',
 'Cytoplasm_Texture_Gabor_ER_5',
 'Cytoplasm_Texture_Gabor_Mito_10',
 'Cytoplasm_Texture_Gabor_Mito_3',
 'Cytoplasm_Texture_Gabor_Mito_5',
 'Cytoplasm_Texture_Gabor_RNA_10',
 'Cytoplasm_Texture_Gabor_RNA_3',
 'Cytoplasm_Texture_Gabor_RNA_5',
 'Cytoplasm_Texture_InfoMeas1_AGP_10_0',
 'Cytoplasm_Texture_InfoMeas1_AGP_3_0',
 'Cytoplasm_Texture_InfoMeas1_AGP_5_0',
 'Cytoplasm_Texture_InfoMeas1_DNA_10_0',
 'Cytoplasm_Texture_InfoMeas1_DNA_3_0',
 'Cytoplasm_Texture_InfoMeas1_DNA_5_0',
 'Cytoplasm_Texture_InfoMeas1_ER_10_0',
 'Cytoplasm_Texture_InfoMeas1_ER_3_0',
 'Cytoplasm_Texture_InfoMeas1_ER_5_0',
 'Cytoplasm_Texture_InfoMeas1_Mito_10_0',
 'Cytoplasm_Texture_InfoMeas1_Mito_3_0',
 'Cytoplasm_Texture_InfoMeas1_Mito_5_0',
 'Cytoplasm_Texture_InfoMeas1_RNA_10_0',
 'Cytoplasm_Texture_InfoMeas1_RNA_3_0',
 'Cytoplasm_Texture_InfoMeas1_RNA_5_0',
 'Cytoplasm_Texture_InfoMeas2_AGP_10_0',
 'Cytoplasm_Texture_InfoMeas2_AGP_3_0',
 'Cytoplasm_Texture_InfoMeas2_AGP_5_0',
 'Cytoplasm_Texture_InfoMeas2_DNA_10_0',
 'Cytoplasm_Texture_InfoMeas2_DNA_3_0',
 'Cytoplasm_Texture_InfoMeas2_DNA_5_0',
 'Cytoplasm_Texture_InfoMeas2_ER_10_0',
 'Cytoplasm_Texture_InfoMeas2_ER_3_0',
 'Cytoplasm_Texture_InfoMeas2_ER_5_0',
 'Cytoplasm_Texture_InfoMeas2_Mito_10_0',
 'Cytoplasm_Texture_InfoMeas2_Mito_3_0',
 'Cytoplasm_Texture_InfoMeas2_Mito_5_0',
 'Cytoplasm_Texture_InfoMeas2_RNA_10_0',
 'Cytoplasm_Texture_InfoMeas2_RNA_3_0',
 'Cytoplasm_Texture_InfoMeas2_RNA_5_0',
 'Cytoplasm_Texture_InverseDifferenceMoment_AGP_10_0',
 'Cytoplasm_Texture_InverseDifferenceMoment_AGP_3_0',
 'Cytoplasm_Texture_InverseDifferenceMoment_AGP_5_0',
 'Cytoplasm_Texture_InverseDifferenceMoment_DNA_10_0',
 'Cytoplasm_Texture_InverseDifferenceMoment_DNA_3_0',
 'Cytoplasm_Texture_InverseDifferenceMoment_DNA_5_0',
 'Cytoplasm_Texture_InverseDifferenceMoment_ER_10_0',
 'Cytoplasm_Texture_InverseDifferenceMoment_ER_3_0',
 'Cytoplasm_Texture_InverseDifferenceMoment_ER_5_0',
 'Cytoplasm_Texture_InverseDifferenceMoment_Mito_10_0',
 'Cytoplasm_Texture_InverseDifferenceMoment_Mito_3_0',
 'Cytoplasm_Texture_InverseDifferenceMoment_Mito_5_0',
 'Cytoplasm_Texture_InverseDifferenceMoment_RNA_10_0',
 'Cytoplasm_Texture_InverseDifferenceMoment_RNA_3_0',
 'Cytoplasm_Texture_InverseDifferenceMoment_RNA_5_0',
 'Cytoplasm_Texture_SumAverage_AGP_10_0',
 'Cytoplasm_Texture_SumAverage_AGP_3_0',
 'Cytoplasm_Texture_SumAverage_AGP_5_0',
 'Cytoplasm_Texture_SumAverage_DNA_10_0',
 'Cytoplasm_Texture_SumAverage_DNA_3_0',
 'Cytoplasm_Texture_SumAverage_DNA_5_0',
 'Cytoplasm_Texture_SumAverage_ER_10_0',
 'Cytoplasm_Texture_SumAverage_ER_3_0',
 'Cytoplasm_Texture_SumAverage_ER_5_0',
 'Cytoplasm_Texture_SumAverage_Mito_10_0',
 'Cytoplasm_Texture_SumAverage_Mito_3_0',
 'Cytoplasm_Texture_SumAverage_Mito_5_0',
 'Cytoplasm_Texture_SumAverage_RNA_10_0',
 'Cytoplasm_Texture_SumAverage_RNA_3_0',
 'Cytoplasm_Texture_SumAverage_RNA_5_0',
 'Cytoplasm_Texture_SumEntropy_AGP_10_0',
 'Cytoplasm_Texture_SumEntropy_AGP_3_0',
 'Cytoplasm_Texture_SumEntropy_AGP_5_0',
 'Cytoplasm_Texture_SumEntropy_DNA_10_0',
 'Cytoplasm_Texture_SumEntropy_DNA_3_0',
 'Cytoplasm_Texture_SumEntropy_DNA_5_0',
 'Cytoplasm_Texture_SumEntropy_ER_10_0',
 'Cytoplasm_Texture_SumEntropy_ER_3_0',
 'Cytoplasm_Texture_SumEntropy_ER_5_0',
 'Cytoplasm_Texture_SumEntropy_Mito_10_0',
 'Cytoplasm_Texture_SumEntropy_Mito_3_0',
 'Cytoplasm_Texture_SumEntropy_Mito_5_0',
 'Cytoplasm_Texture_SumEntropy_RNA_10_0',
 'Cytoplasm_Texture_SumEntropy_RNA_3_0',
 'Cytoplasm_Texture_SumEntropy_RNA_5_0',
 'Cytoplasm_Texture_SumVariance_AGP_10_0',
 'Cytoplasm_Texture_SumVariance_AGP_3_0',
 'Cytoplasm_Texture_SumVariance_AGP_5_0',
 'Cytoplasm_Texture_SumVariance_DNA_10_0',
 'Cytoplasm_Texture_SumVariance_DNA_3_0',
 'Cytoplasm_Texture_SumVariance_DNA_5_0',
 'Cytoplasm_Texture_SumVariance_ER_10_0',
 'Cytoplasm_Texture_SumVariance_ER_3_0',
 'Cytoplasm_Texture_SumVariance_ER_5_0',
 'Cytoplasm_Texture_SumVariance_Mito_10_0',
 'Cytoplasm_Texture_SumVariance_Mito_3_0',
 'Cytoplasm_Texture_SumVariance_Mito_5_0',
 'Cytoplasm_Texture_SumVariance_RNA_10_0',
 'Cytoplasm_Texture_SumVariance_RNA_3_0',
 'Cytoplasm_Texture_SumVariance_RNA_5_0',
 'Cytoplasm_Texture_Variance_AGP_10_0',
 'Cytoplasm_Texture_Variance_AGP_3_0',
 'Cytoplasm_Texture_Variance_AGP_5_0',
 'Cytoplasm_Texture_Variance_DNA_10_0',
 'Cytoplasm_Texture_Variance_DNA_3_0',
 'Cytoplasm_Texture_Variance_DNA_5_0',
 'Cytoplasm_Texture_Variance_ER_10_0',
 'Cytoplasm_Texture_Variance_ER_3_0',
 'Cytoplasm_Texture_Variance_ER_5_0',
 'Cytoplasm_Texture_Variance_Mito_10_0',
 'Cytoplasm_Texture_Variance_Mito_3_0',
 'Cytoplasm_Texture_Variance_Mito_5_0',
 'Cytoplasm_Texture_Variance_RNA_10_0',
 'Cytoplasm_Texture_Variance_RNA_3_0',
 'Cytoplasm_Texture_Variance_RNA_5_0'
</p>
</details>

<details><summary>`Nuclei` Table</summary>
<p>
'TableNumber',
 'ImageNumber',
 'ObjectNumber',
 'Nuclei_AreaShape_Area',
 'Nuclei_AreaShape_Center_X',
 'Nuclei_AreaShape_Center_Y',
 'Nuclei_AreaShape_Compactness',
 'Nuclei_AreaShape_Eccentricity',
 'Nuclei_AreaShape_EulerNumber',
 'Nuclei_AreaShape_Extent',
 'Nuclei_AreaShape_FormFactor',
 'Nuclei_AreaShape_MajorAxisLength',
 'Nuclei_AreaShape_MaxFeretDiameter',
 'Nuclei_AreaShape_MaximumRadius',
 'Nuclei_AreaShape_MeanRadius',
 'Nuclei_AreaShape_MedianRadius',
 'Nuclei_AreaShape_MinFeretDiameter',
 'Nuclei_AreaShape_MinorAxisLength',
 'Nuclei_AreaShape_Orientation',
 'Nuclei_AreaShape_Perimeter',
 'Nuclei_AreaShape_Solidity',
 'Nuclei_AreaShape_Zernike_0_0',
 'Nuclei_AreaShape_Zernike_1_1',
 'Nuclei_AreaShape_Zernike_2_0',
 'Nuclei_AreaShape_Zernike_2_2',
 'Nuclei_AreaShape_Zernike_3_1',
 'Nuclei_AreaShape_Zernike_3_3',
 'Nuclei_AreaShape_Zernike_4_0',
 'Nuclei_AreaShape_Zernike_4_2',
 'Nuclei_AreaShape_Zernike_4_4',
 'Nuclei_AreaShape_Zernike_5_1',
 'Nuclei_AreaShape_Zernike_5_3',
 'Nuclei_AreaShape_Zernike_5_5',
 'Nuclei_AreaShape_Zernike_6_0',
 'Nuclei_AreaShape_Zernike_6_2',
 'Nuclei_AreaShape_Zernike_6_4',
 'Nuclei_AreaShape_Zernike_6_6',
 'Nuclei_AreaShape_Zernike_7_1',
 'Nuclei_AreaShape_Zernike_7_3',
 'Nuclei_AreaShape_Zernike_7_5',
 'Nuclei_AreaShape_Zernike_7_7',
 'Nuclei_AreaShape_Zernike_8_0',
 'Nuclei_AreaShape_Zernike_8_2',
 'Nuclei_AreaShape_Zernike_8_4',
 'Nuclei_AreaShape_Zernike_8_6',
 'Nuclei_AreaShape_Zernike_8_8',
 'Nuclei_AreaShape_Zernike_9_1',
 'Nuclei_AreaShape_Zernike_9_3',
 'Nuclei_AreaShape_Zernike_9_5',
 'Nuclei_AreaShape_Zernike_9_7',
 'Nuclei_AreaShape_Zernike_9_9',
 'Nuclei_Children_Cells_Count',
 'Nuclei_Children_Cytoplasm_Count',
 'Nuclei_Correlation_Correlation_DNA_AGP',
 'Nuclei_Correlation_Correlation_DNA_ER',
 'Nuclei_Correlation_Correlation_DNA_Mito',
 'Nuclei_Correlation_Correlation_DNA_RNA',
 'Nuclei_Correlation_Correlation_ER_AGP',
 'Nuclei_Correlation_Correlation_ER_RNA',
 'Nuclei_Correlation_Correlation_Mito_AGP',
 'Nuclei_Correlation_Correlation_Mito_ER',
 'Nuclei_Correlation_Correlation_Mito_RNA',
 'Nuclei_Correlation_Correlation_RNA_AGP',
 'Nuclei_Correlation_Costes_AGP_DNA',
 'Nuclei_Correlation_Costes_AGP_ER',
 'Nuclei_Correlation_Costes_AGP_Mito',
 'Nuclei_Correlation_Costes_AGP_RNA',
 'Nuclei_Correlation_Costes_DNA_AGP',
 'Nuclei_Correlation_Costes_DNA_ER',
 'Nuclei_Correlation_Costes_DNA_Mito',
 'Nuclei_Correlation_Costes_DNA_RNA',
 'Nuclei_Correlation_Costes_ER_AGP',
 'Nuclei_Correlation_Costes_ER_DNA',
 'Nuclei_Correlation_Costes_ER_Mito',
 'Nuclei_Correlation_Costes_ER_RNA',
 'Nuclei_Correlation_Costes_Mito_AGP',
 'Nuclei_Correlation_Costes_Mito_DNA',
 'Nuclei_Correlation_Costes_Mito_ER',
 'Nuclei_Correlation_Costes_Mito_RNA',
 'Nuclei_Correlation_Costes_RNA_AGP',
 'Nuclei_Correlation_Costes_RNA_DNA',
 'Nuclei_Correlation_Costes_RNA_ER',
 'Nuclei_Correlation_Costes_RNA_Mito',
 'Nuclei_Correlation_K_AGP_DNA',
 'Nuclei_Correlation_K_AGP_ER',
 'Nuclei_Correlation_K_AGP_Mito',
 'Nuclei_Correlation_K_AGP_RNA',
 'Nuclei_Correlation_K_DNA_AGP',
 'Nuclei_Correlation_K_DNA_ER',
 'Nuclei_Correlation_K_DNA_Mito',
 'Nuclei_Correlation_K_DNA_RNA',
 'Nuclei_Correlation_K_ER_AGP',
 'Nuclei_Correlation_K_ER_DNA',
 'Nuclei_Correlation_K_ER_Mito',
 'Nuclei_Correlation_K_ER_RNA',
 'Nuclei_Correlation_K_Mito_AGP',
 'Nuclei_Correlation_K_Mito_DNA',
 'Nuclei_Correlation_K_Mito_ER',
 'Nuclei_Correlation_K_Mito_RNA',
 'Nuclei_Correlation_K_RNA_AGP',
 'Nuclei_Correlation_K_RNA_DNA',
 'Nuclei_Correlation_K_RNA_ER',
 'Nuclei_Correlation_K_RNA_Mito',
 'Nuclei_Correlation_Manders_AGP_DNA',
 'Nuclei_Correlation_Manders_AGP_ER',
 'Nuclei_Correlation_Manders_AGP_Mito',
 'Nuclei_Correlation_Manders_AGP_RNA',
 'Nuclei_Correlation_Manders_DNA_AGP',
 'Nuclei_Correlation_Manders_DNA_ER',
 'Nuclei_Correlation_Manders_DNA_Mito',
 'Nuclei_Correlation_Manders_DNA_RNA',
 'Nuclei_Correlation_Manders_ER_AGP',
 'Nuclei_Correlation_Manders_ER_DNA',
 'Nuclei_Correlation_Manders_ER_Mito',
 'Nuclei_Correlation_Manders_ER_RNA',
 'Nuclei_Correlation_Manders_Mito_AGP',
 'Nuclei_Correlation_Manders_Mito_DNA',
 'Nuclei_Correlation_Manders_Mito_ER',
 'Nuclei_Correlation_Manders_Mito_RNA',
 'Nuclei_Correlation_Manders_RNA_AGP',
 'Nuclei_Correlation_Manders_RNA_DNA',
 'Nuclei_Correlation_Manders_RNA_ER',
 'Nuclei_Correlation_Manders_RNA_Mito',
 'Nuclei_Correlation_Overlap_DNA_AGP',
 'Nuclei_Correlation_Overlap_DNA_ER',
 'Nuclei_Correlation_Overlap_DNA_Mito',
 'Nuclei_Correlation_Overlap_DNA_RNA',
 'Nuclei_Correlation_Overlap_ER_AGP',
 'Nuclei_Correlation_Overlap_ER_RNA',
 'Nuclei_Correlation_Overlap_Mito_AGP',
 'Nuclei_Correlation_Overlap_Mito_ER',
 'Nuclei_Correlation_Overlap_Mito_RNA',
 'Nuclei_Correlation_Overlap_RNA_AGP',
 'Nuclei_Correlation_RWC_AGP_DNA',
 'Nuclei_Correlation_RWC_AGP_ER',
 'Nuclei_Correlation_RWC_AGP_Mito',
 'Nuclei_Correlation_RWC_AGP_RNA',
 'Nuclei_Correlation_RWC_DNA_AGP',
 'Nuclei_Correlation_RWC_DNA_ER',
 'Nuclei_Correlation_RWC_DNA_Mito',
 'Nuclei_Correlation_RWC_DNA_RNA',
 'Nuclei_Correlation_RWC_ER_AGP',
 'Nuclei_Correlation_RWC_ER_DNA',
 'Nuclei_Correlation_RWC_ER_Mito',
 'Nuclei_Correlation_RWC_ER_RNA',
 'Nuclei_Correlation_RWC_Mito_AGP',
 'Nuclei_Correlation_RWC_Mito_DNA',
 'Nuclei_Correlation_RWC_Mito_ER',
 'Nuclei_Correlation_RWC_Mito_RNA',
 'Nuclei_Correlation_RWC_RNA_AGP',
 'Nuclei_Correlation_RWC_RNA_DNA',
 'Nuclei_Correlation_RWC_RNA_ER',
 'Nuclei_Correlation_RWC_RNA_Mito',
 'Nuclei_Granularity_10_AGP',
 'Nuclei_Granularity_10_DNA',
 'Nuclei_Granularity_10_ER',
 'Nuclei_Granularity_10_Mito',
 'Nuclei_Granularity_10_RNA',
 'Nuclei_Granularity_11_AGP',
 'Nuclei_Granularity_11_DNA',
 'Nuclei_Granularity_11_ER',
 'Nuclei_Granularity_11_Mito',
 'Nuclei_Granularity_11_RNA',
 'Nuclei_Granularity_12_AGP',
 'Nuclei_Granularity_12_DNA',
 'Nuclei_Granularity_12_ER',
 'Nuclei_Granularity_12_Mito',
 'Nuclei_Granularity_12_RNA',
 'Nuclei_Granularity_13_AGP',
 'Nuclei_Granularity_13_DNA',
 'Nuclei_Granularity_13_ER',
 'Nuclei_Granularity_13_Mito',
 'Nuclei_Granularity_13_RNA',
 'Nuclei_Granularity_14_AGP',
 'Nuclei_Granularity_14_DNA',
 'Nuclei_Granularity_14_ER',
 'Nuclei_Granularity_14_Mito',
 'Nuclei_Granularity_14_RNA',
 'Nuclei_Granularity_15_AGP',
 'Nuclei_Granularity_15_DNA',
 'Nuclei_Granularity_15_ER',
 'Nuclei_Granularity_15_Mito',
 'Nuclei_Granularity_15_RNA',
 'Nuclei_Granularity_16_AGP',
 'Nuclei_Granularity_16_DNA',
 'Nuclei_Granularity_16_ER',
 'Nuclei_Granularity_16_Mito',
 'Nuclei_Granularity_16_RNA',
 'Nuclei_Granularity_1_AGP',
 'Nuclei_Granularity_1_DNA',
 'Nuclei_Granularity_1_ER',
 'Nuclei_Granularity_1_Mito',
 'Nuclei_Granularity_1_RNA',
 'Nuclei_Granularity_2_AGP',
 'Nuclei_Granularity_2_DNA',
 'Nuclei_Granularity_2_ER',
 'Nuclei_Granularity_2_Mito',
 'Nuclei_Granularity_2_RNA',
 'Nuclei_Granularity_3_AGP',
 'Nuclei_Granularity_3_DNA',
 'Nuclei_Granularity_3_ER',
 'Nuclei_Granularity_3_Mito',
 'Nuclei_Granularity_3_RNA',
 'Nuclei_Granularity_4_AGP',
 'Nuclei_Granularity_4_DNA',
 'Nuclei_Granularity_4_ER',
 'Nuclei_Granularity_4_Mito',
 'Nuclei_Granularity_4_RNA',
 'Nuclei_Granularity_5_AGP',
 'Nuclei_Granularity_5_DNA',
 'Nuclei_Granularity_5_ER',
 'Nuclei_Granularity_5_Mito',
 'Nuclei_Granularity_5_RNA',
 'Nuclei_Granularity_6_AGP',
 'Nuclei_Granularity_6_DNA',
 'Nuclei_Granularity_6_ER',
 'Nuclei_Granularity_6_Mito',
 'Nuclei_Granularity_6_RNA',
 'Nuclei_Granularity_7_AGP',
 'Nuclei_Granularity_7_DNA',
 'Nuclei_Granularity_7_ER',
 'Nuclei_Granularity_7_Mito',
 'Nuclei_Granularity_7_RNA',
 'Nuclei_Granularity_8_AGP',
 'Nuclei_Granularity_8_DNA',
 'Nuclei_Granularity_8_ER',
 'Nuclei_Granularity_8_Mito',
 'Nuclei_Granularity_8_RNA',
 'Nuclei_Granularity_9_AGP',
 'Nuclei_Granularity_9_DNA',
 'Nuclei_Granularity_9_ER',
 'Nuclei_Granularity_9_Mito',
 'Nuclei_Granularity_9_RNA',
 'Nuclei_Intensity_IntegratedIntensityEdge_AGP',
 'Nuclei_Intensity_IntegratedIntensityEdge_DNA',
 'Nuclei_Intensity_IntegratedIntensityEdge_ER',
 'Nuclei_Intensity_IntegratedIntensityEdge_Mito',
 'Nuclei_Intensity_IntegratedIntensityEdge_RNA',
 'Nuclei_Intensity_IntegratedIntensity_AGP',
 'Nuclei_Intensity_IntegratedIntensity_DNA',
 'Nuclei_Intensity_IntegratedIntensity_ER',
 'Nuclei_Intensity_IntegratedIntensity_Mito',
 'Nuclei_Intensity_IntegratedIntensity_RNA',
 'Nuclei_Intensity_LowerQuartileIntensity_AGP',
 'Nuclei_Intensity_LowerQuartileIntensity_DNA',
 'Nuclei_Intensity_LowerQuartileIntensity_ER',
 'Nuclei_Intensity_LowerQuartileIntensity_Mito',
 'Nuclei_Intensity_LowerQuartileIntensity_RNA',
 'Nuclei_Intensity_MADIntensity_AGP',
 'Nuclei_Intensity_MADIntensity_DNA',
 'Nuclei_Intensity_MADIntensity_ER',
 'Nuclei_Intensity_MADIntensity_Mito',
 'Nuclei_Intensity_MADIntensity_RNA',
 'Nuclei_Intensity_MassDisplacement_AGP',
 'Nuclei_Intensity_MassDisplacement_DNA',
 'Nuclei_Intensity_MassDisplacement_ER',
 'Nuclei_Intensity_MassDisplacement_Mito',
 'Nuclei_Intensity_MassDisplacement_RNA',
 'Nuclei_Intensity_MaxIntensityEdge_AGP',
 'Nuclei_Intensity_MaxIntensityEdge_DNA',
 'Nuclei_Intensity_MaxIntensityEdge_ER',
 'Nuclei_Intensity_MaxIntensityEdge_Mito',
 'Nuclei_Intensity_MaxIntensityEdge_RNA',
 'Nuclei_Intensity_MaxIntensity_AGP',
 'Nuclei_Intensity_MaxIntensity_DNA',
 'Nuclei_Intensity_MaxIntensity_ER',
 'Nuclei_Intensity_MaxIntensity_Mito',
 'Nuclei_Intensity_MaxIntensity_RNA',
 'Nuclei_Intensity_MeanIntensityEdge_AGP',
 'Nuclei_Intensity_MeanIntensityEdge_DNA',
 'Nuclei_Intensity_MeanIntensityEdge_ER',
 'Nuclei_Intensity_MeanIntensityEdge_Mito',
 'Nuclei_Intensity_MeanIntensityEdge_RNA',
 'Nuclei_Intensity_MeanIntensity_AGP',
 'Nuclei_Intensity_MeanIntensity_DNA',
 'Nuclei_Intensity_MeanIntensity_ER',
 'Nuclei_Intensity_MeanIntensity_Mito',
 'Nuclei_Intensity_MeanIntensity_RNA',
 'Nuclei_Intensity_MedianIntensity_AGP',
 'Nuclei_Intensity_MedianIntensity_DNA',
 'Nuclei_Intensity_MedianIntensity_ER',
 'Nuclei_Intensity_MedianIntensity_Mito',
 'Nuclei_Intensity_MedianIntensity_RNA',
 'Nuclei_Intensity_MinIntensityEdge_AGP',
 'Nuclei_Intensity_MinIntensityEdge_DNA',
 'Nuclei_Intensity_MinIntensityEdge_ER',
 'Nuclei_Intensity_MinIntensityEdge_Mito',
 'Nuclei_Intensity_MinIntensityEdge_RNA',
 'Nuclei_Intensity_MinIntensity_AGP',
 'Nuclei_Intensity_MinIntensity_DNA',
 'Nuclei_Intensity_MinIntensity_ER',
 'Nuclei_Intensity_MinIntensity_Mito',
 'Nuclei_Intensity_MinIntensity_RNA',
 'Nuclei_Intensity_StdIntensityEdge_AGP',
 'Nuclei_Intensity_StdIntensityEdge_DNA',
 'Nuclei_Intensity_StdIntensityEdge_ER',
 'Nuclei_Intensity_StdIntensityEdge_Mito',
 'Nuclei_Intensity_StdIntensityEdge_RNA',
 'Nuclei_Intensity_StdIntensity_AGP',
 'Nuclei_Intensity_StdIntensity_DNA',
 'Nuclei_Intensity_StdIntensity_ER',
 'Nuclei_Intensity_StdIntensity_Mito',
 'Nuclei_Intensity_StdIntensity_RNA',
 'Nuclei_Intensity_UpperQuartileIntensity_AGP',
 'Nuclei_Intensity_UpperQuartileIntensity_DNA',
 'Nuclei_Intensity_UpperQuartileIntensity_ER',
 'Nuclei_Intensity_UpperQuartileIntensity_Mito',
 'Nuclei_Intensity_UpperQuartileIntensity_RNA',
 'Nuclei_Location_CenterMassIntensity_X_AGP',
 'Nuclei_Location_CenterMassIntensity_X_DNA',
 'Nuclei_Location_CenterMassIntensity_X_ER',
 'Nuclei_Location_CenterMassIntensity_X_Mito',
 'Nuclei_Location_CenterMassIntensity_X_RNA',
 'Nuclei_Location_CenterMassIntensity_Y_AGP',
 'Nuclei_Location_CenterMassIntensity_Y_DNA',
 'Nuclei_Location_CenterMassIntensity_Y_ER',
 'Nuclei_Location_CenterMassIntensity_Y_Mito',
 'Nuclei_Location_CenterMassIntensity_Y_RNA',
 'Nuclei_Location_Center_X',
 'Nuclei_Location_Center_Y',
 'Nuclei_Location_MaxIntensity_X_AGP',
 'Nuclei_Location_MaxIntensity_X_DNA',
 'Nuclei_Location_MaxIntensity_X_ER',
 'Nuclei_Location_MaxIntensity_X_Mito',
 'Nuclei_Location_MaxIntensity_X_RNA',
 'Nuclei_Location_MaxIntensity_Y_AGP',
 'Nuclei_Location_MaxIntensity_Y_DNA',
 'Nuclei_Location_MaxIntensity_Y_ER',
 'Nuclei_Location_MaxIntensity_Y_Mito',
 'Nuclei_Location_MaxIntensity_Y_RNA',
 'Nuclei_Neighbors_AngleBetweenNeighbors_1',
 'Nuclei_Neighbors_FirstClosestDistance_1',
 'Nuclei_Neighbors_FirstClosestObjectNumber_1',
 'Nuclei_Neighbors_NumberOfNeighbors_1',
 'Nuclei_Neighbors_PercentTouching_1',
 'Nuclei_Neighbors_SecondClosestDistance_1',
 'Nuclei_Neighbors_SecondClosestObjectNumber_1',
 'Nuclei_Number_Object_Number',
 'Nuclei_RadialDistribution_FracAtD_AGP_1of4',
 'Nuclei_RadialDistribution_FracAtD_AGP_2of4',
 'Nuclei_RadialDistribution_FracAtD_AGP_3of4',
 'Nuclei_RadialDistribution_FracAtD_AGP_4of4',
 'Nuclei_RadialDistribution_FracAtD_DNA_1of4',
 'Nuclei_RadialDistribution_FracAtD_DNA_2of4',
 'Nuclei_RadialDistribution_FracAtD_DNA_3of4',
 'Nuclei_RadialDistribution_FracAtD_DNA_4of4',
 'Nuclei_RadialDistribution_FracAtD_ER_1of4',
 'Nuclei_RadialDistribution_FracAtD_ER_2of4',
 'Nuclei_RadialDistribution_FracAtD_ER_3of4',
 'Nuclei_RadialDistribution_FracAtD_ER_4of4',
 'Nuclei_RadialDistribution_FracAtD_Mito_1of4',
 'Nuclei_RadialDistribution_FracAtD_Mito_2of4',
 'Nuclei_RadialDistribution_FracAtD_Mito_3of4',
 'Nuclei_RadialDistribution_FracAtD_Mito_4of4',
 'Nuclei_RadialDistribution_FracAtD_RNA_1of4',
 'Nuclei_RadialDistribution_FracAtD_RNA_2of4',
 'Nuclei_RadialDistribution_FracAtD_RNA_3of4',
 'Nuclei_RadialDistribution_FracAtD_RNA_4of4',
 'Nuclei_RadialDistribution_MeanFrac_AGP_1of4',
 'Nuclei_RadialDistribution_MeanFrac_AGP_2of4',
 'Nuclei_RadialDistribution_MeanFrac_AGP_3of4',
 'Nuclei_RadialDistribution_MeanFrac_AGP_4of4',
 'Nuclei_RadialDistribution_MeanFrac_DNA_1of4',
 'Nuclei_RadialDistribution_MeanFrac_DNA_2of4',
 'Nuclei_RadialDistribution_MeanFrac_DNA_3of4',
 'Nuclei_RadialDistribution_MeanFrac_DNA_4of4',
 'Nuclei_RadialDistribution_MeanFrac_ER_1of4',
 'Nuclei_RadialDistribution_MeanFrac_ER_2of4',
 'Nuclei_RadialDistribution_MeanFrac_ER_3of4',
 'Nuclei_RadialDistribution_MeanFrac_ER_4of4',
 'Nuclei_RadialDistribution_MeanFrac_Mito_1of4',
 'Nuclei_RadialDistribution_MeanFrac_Mito_2of4',
 'Nuclei_RadialDistribution_MeanFrac_Mito_3of4',
 'Nuclei_RadialDistribution_MeanFrac_Mito_4of4',
 'Nuclei_RadialDistribution_MeanFrac_RNA_1of4',
 'Nuclei_RadialDistribution_MeanFrac_RNA_2of4',
 'Nuclei_RadialDistribution_MeanFrac_RNA_3of4',
 'Nuclei_RadialDistribution_MeanFrac_RNA_4of4',
 'Nuclei_RadialDistribution_RadialCV_AGP_1of4',
 'Nuclei_RadialDistribution_RadialCV_AGP_2of4',
 'Nuclei_RadialDistribution_RadialCV_AGP_3of4',
 'Nuclei_RadialDistribution_RadialCV_AGP_4of4',
 'Nuclei_RadialDistribution_RadialCV_DNA_1of4',
 'Nuclei_RadialDistribution_RadialCV_DNA_2of4',
 'Nuclei_RadialDistribution_RadialCV_DNA_3of4',
 'Nuclei_RadialDistribution_RadialCV_DNA_4of4',
 'Nuclei_RadialDistribution_RadialCV_ER_1of4',
 'Nuclei_RadialDistribution_RadialCV_ER_2of4',
 'Nuclei_RadialDistribution_RadialCV_ER_3of4',
 'Nuclei_RadialDistribution_RadialCV_ER_4of4',
 'Nuclei_RadialDistribution_RadialCV_Mito_1of4',
 'Nuclei_RadialDistribution_RadialCV_Mito_2of4',
 'Nuclei_RadialDistribution_RadialCV_Mito_3of4',
 'Nuclei_RadialDistribution_RadialCV_Mito_4of4',
 'Nuclei_RadialDistribution_RadialCV_RNA_1of4',
 'Nuclei_RadialDistribution_RadialCV_RNA_2of4',
 'Nuclei_RadialDistribution_RadialCV_RNA_3of4',
 'Nuclei_RadialDistribution_RadialCV_RNA_4of4',
 'Nuclei_Texture_AngularSecondMoment_AGP_10_0',
 'Nuclei_Texture_AngularSecondMoment_AGP_3_0',
 'Nuclei_Texture_AngularSecondMoment_AGP_5_0',
 'Nuclei_Texture_AngularSecondMoment_DNA_10_0',
 'Nuclei_Texture_AngularSecondMoment_DNA_3_0',
 'Nuclei_Texture_AngularSecondMoment_DNA_5_0',
 'Nuclei_Texture_AngularSecondMoment_ER_10_0',
 'Nuclei_Texture_AngularSecondMoment_ER_3_0',
 'Nuclei_Texture_AngularSecondMoment_ER_5_0',
 'Nuclei_Texture_AngularSecondMoment_Mito_10_0',
 'Nuclei_Texture_AngularSecondMoment_Mito_3_0',
 'Nuclei_Texture_AngularSecondMoment_Mito_5_0',
 'Nuclei_Texture_AngularSecondMoment_RNA_10_0',
 'Nuclei_Texture_AngularSecondMoment_RNA_3_0',
 'Nuclei_Texture_AngularSecondMoment_RNA_5_0',
 'Nuclei_Texture_Contrast_AGP_10_0',
 'Nuclei_Texture_Contrast_AGP_3_0',
 'Nuclei_Texture_Contrast_AGP_5_0',
 'Nuclei_Texture_Contrast_DNA_10_0',
 'Nuclei_Texture_Contrast_DNA_3_0',
 'Nuclei_Texture_Contrast_DNA_5_0',
 'Nuclei_Texture_Contrast_ER_10_0',
 'Nuclei_Texture_Contrast_ER_3_0',
 'Nuclei_Texture_Contrast_ER_5_0',
 'Nuclei_Texture_Contrast_Mito_10_0',
 'Nuclei_Texture_Contrast_Mito_3_0',
 'Nuclei_Texture_Contrast_Mito_5_0',
 'Nuclei_Texture_Contrast_RNA_10_0',
 'Nuclei_Texture_Contrast_RNA_3_0',
 'Nuclei_Texture_Contrast_RNA_5_0',
 'Nuclei_Texture_Correlation_AGP_10_0',
 'Nuclei_Texture_Correlation_AGP_3_0',
 'Nuclei_Texture_Correlation_AGP_5_0',
 'Nuclei_Texture_Correlation_DNA_10_0',
 'Nuclei_Texture_Correlation_DNA_3_0',
 'Nuclei_Texture_Correlation_DNA_5_0',
 'Nuclei_Texture_Correlation_ER_10_0',
 'Nuclei_Texture_Correlation_ER_3_0',
 'Nuclei_Texture_Correlation_ER_5_0',
 'Nuclei_Texture_Correlation_Mito_10_0',
 'Nuclei_Texture_Correlation_Mito_3_0',
 'Nuclei_Texture_Correlation_Mito_5_0',
 'Nuclei_Texture_Correlation_RNA_10_0',
 'Nuclei_Texture_Correlation_RNA_3_0',
 'Nuclei_Texture_Correlation_RNA_5_0',
 'Nuclei_Texture_DifferenceEntropy_AGP_10_0',
 'Nuclei_Texture_DifferenceEntropy_AGP_3_0',
 'Nuclei_Texture_DifferenceEntropy_AGP_5_0',
 'Nuclei_Texture_DifferenceEntropy_DNA_10_0',
 'Nuclei_Texture_DifferenceEntropy_DNA_3_0',
 'Nuclei_Texture_DifferenceEntropy_DNA_5_0',
 'Nuclei_Texture_DifferenceEntropy_ER_10_0',
 'Nuclei_Texture_DifferenceEntropy_ER_3_0',
 'Nuclei_Texture_DifferenceEntropy_ER_5_0',
 'Nuclei_Texture_DifferenceEntropy_Mito_10_0',
 'Nuclei_Texture_DifferenceEntropy_Mito_3_0',
 'Nuclei_Texture_DifferenceEntropy_Mito_5_0',
 'Nuclei_Texture_DifferenceEntropy_RNA_10_0',
 'Nuclei_Texture_DifferenceEntropy_RNA_3_0',
 'Nuclei_Texture_DifferenceEntropy_RNA_5_0',
 'Nuclei_Texture_DifferenceVariance_AGP_10_0',
 'Nuclei_Texture_DifferenceVariance_AGP_3_0',
 'Nuclei_Texture_DifferenceVariance_AGP_5_0',
 'Nuclei_Texture_DifferenceVariance_DNA_10_0',
 'Nuclei_Texture_DifferenceVariance_DNA_3_0',
 'Nuclei_Texture_DifferenceVariance_DNA_5_0',
 'Nuclei_Texture_DifferenceVariance_ER_10_0',
 'Nuclei_Texture_DifferenceVariance_ER_3_0',
 'Nuclei_Texture_DifferenceVariance_ER_5_0',
 'Nuclei_Texture_DifferenceVariance_Mito_10_0',
 'Nuclei_Texture_DifferenceVariance_Mito_3_0',
 'Nuclei_Texture_DifferenceVariance_Mito_5_0',
 'Nuclei_Texture_DifferenceVariance_RNA_10_0',
 'Nuclei_Texture_DifferenceVariance_RNA_3_0',
 'Nuclei_Texture_DifferenceVariance_RNA_5_0',
 'Nuclei_Texture_Entropy_AGP_10_0',
 'Nuclei_Texture_Entropy_AGP_3_0',
 'Nuclei_Texture_Entropy_AGP_5_0',
 'Nuclei_Texture_Entropy_DNA_10_0',
 'Nuclei_Texture_Entropy_DNA_3_0',
 'Nuclei_Texture_Entropy_DNA_5_0',
 'Nuclei_Texture_Entropy_ER_10_0',
 'Nuclei_Texture_Entropy_ER_3_0',
 'Nuclei_Texture_Entropy_ER_5_0',
 'Nuclei_Texture_Entropy_Mito_10_0',
 'Nuclei_Texture_Entropy_Mito_3_0',
 'Nuclei_Texture_Entropy_Mito_5_0',
 'Nuclei_Texture_Entropy_RNA_10_0',
 'Nuclei_Texture_Entropy_RNA_3_0',
 'Nuclei_Texture_Entropy_RNA_5_0',
 'Nuclei_Texture_Gabor_AGP_10',
 'Nuclei_Texture_Gabor_AGP_3',
 'Nuclei_Texture_Gabor_AGP_5',
 'Nuclei_Texture_Gabor_DNA_10',
 'Nuclei_Texture_Gabor_DNA_3',
 'Nuclei_Texture_Gabor_DNA_5',
 'Nuclei_Texture_Gabor_ER_10',
 'Nuclei_Texture_Gabor_ER_3',
 'Nuclei_Texture_Gabor_ER_5',
 'Nuclei_Texture_Gabor_Mito_10',
 'Nuclei_Texture_Gabor_Mito_3',
 'Nuclei_Texture_Gabor_Mito_5',
 'Nuclei_Texture_Gabor_RNA_10',
 'Nuclei_Texture_Gabor_RNA_3',
 'Nuclei_Texture_Gabor_RNA_5',
 'Nuclei_Texture_InfoMeas1_AGP_10_0',
 'Nuclei_Texture_InfoMeas1_AGP_3_0',
 'Nuclei_Texture_InfoMeas1_AGP_5_0',
 'Nuclei_Texture_InfoMeas1_DNA_10_0',
 'Nuclei_Texture_InfoMeas1_DNA_3_0',
 'Nuclei_Texture_InfoMeas1_DNA_5_0',
 'Nuclei_Texture_InfoMeas1_ER_10_0',
 'Nuclei_Texture_InfoMeas1_ER_3_0',
 'Nuclei_Texture_InfoMeas1_ER_5_0',
 'Nuclei_Texture_InfoMeas1_Mito_10_0',
 'Nuclei_Texture_InfoMeas1_Mito_3_0',
 'Nuclei_Texture_InfoMeas1_Mito_5_0',
 'Nuclei_Texture_InfoMeas1_RNA_10_0',
 'Nuclei_Texture_InfoMeas1_RNA_3_0',
 'Nuclei_Texture_InfoMeas1_RNA_5_0',
 'Nuclei_Texture_InfoMeas2_AGP_10_0',
 'Nuclei_Texture_InfoMeas2_AGP_3_0',
 'Nuclei_Texture_InfoMeas2_AGP_5_0',
 'Nuclei_Texture_InfoMeas2_DNA_10_0',
 'Nuclei_Texture_InfoMeas2_DNA_3_0',
 'Nuclei_Texture_InfoMeas2_DNA_5_0',
 'Nuclei_Texture_InfoMeas2_ER_10_0',
 'Nuclei_Texture_InfoMeas2_ER_3_0',
 'Nuclei_Texture_InfoMeas2_ER_5_0',
 'Nuclei_Texture_InfoMeas2_Mito_10_0',
 'Nuclei_Texture_InfoMeas2_Mito_3_0',
 'Nuclei_Texture_InfoMeas2_Mito_5_0',
 'Nuclei_Texture_InfoMeas2_RNA_10_0',
 'Nuclei_Texture_InfoMeas2_RNA_3_0',
 'Nuclei_Texture_InfoMeas2_RNA_5_0',
 'Nuclei_Texture_InverseDifferenceMoment_AGP_10_0',
 'Nuclei_Texture_InverseDifferenceMoment_AGP_3_0',
 'Nuclei_Texture_InverseDifferenceMoment_AGP_5_0',
 'Nuclei_Texture_InverseDifferenceMoment_DNA_10_0',
 'Nuclei_Texture_InverseDifferenceMoment_DNA_3_0',
 'Nuclei_Texture_InverseDifferenceMoment_DNA_5_0',
 'Nuclei_Texture_InverseDifferenceMoment_ER_10_0',
 'Nuclei_Texture_InverseDifferenceMoment_ER_3_0',
 'Nuclei_Texture_InverseDifferenceMoment_ER_5_0',
 'Nuclei_Texture_InverseDifferenceMoment_Mito_10_0',
 'Nuclei_Texture_InverseDifferenceMoment_Mito_3_0',
 'Nuclei_Texture_InverseDifferenceMoment_Mito_5_0',
 'Nuclei_Texture_InverseDifferenceMoment_RNA_10_0',
 'Nuclei_Texture_InverseDifferenceMoment_RNA_3_0',
 'Nuclei_Texture_InverseDifferenceMoment_RNA_5_0',
 'Nuclei_Texture_SumAverage_AGP_10_0',
 'Nuclei_Texture_SumAverage_AGP_3_0',
 'Nuclei_Texture_SumAverage_AGP_5_0',
 'Nuclei_Texture_SumAverage_DNA_10_0',
 'Nuclei_Texture_SumAverage_DNA_3_0',
 'Nuclei_Texture_SumAverage_DNA_5_0',
 'Nuclei_Texture_SumAverage_ER_10_0',
 'Nuclei_Texture_SumAverage_ER_3_0',
 'Nuclei_Texture_SumAverage_ER_5_0',
 'Nuclei_Texture_SumAverage_Mito_10_0',
 'Nuclei_Texture_SumAverage_Mito_3_0',
 'Nuclei_Texture_SumAverage_Mito_5_0',
 'Nuclei_Texture_SumAverage_RNA_10_0',
 'Nuclei_Texture_SumAverage_RNA_3_0',
 'Nuclei_Texture_SumAverage_RNA_5_0',
 'Nuclei_Texture_SumEntropy_AGP_10_0',
 'Nuclei_Texture_SumEntropy_AGP_3_0',
 'Nuclei_Texture_SumEntropy_AGP_5_0',
 'Nuclei_Texture_SumEntropy_DNA_10_0',
 'Nuclei_Texture_SumEntropy_DNA_3_0',
 'Nuclei_Texture_SumEntropy_DNA_5_0',
 'Nuclei_Texture_SumEntropy_ER_10_0',
 'Nuclei_Texture_SumEntropy_ER_3_0',
 'Nuclei_Texture_SumEntropy_ER_5_0',
 'Nuclei_Texture_SumEntropy_Mito_10_0',
 'Nuclei_Texture_SumEntropy_Mito_3_0',
 'Nuclei_Texture_SumEntropy_Mito_5_0',
 'Nuclei_Texture_SumEntropy_RNA_10_0',
 'Nuclei_Texture_SumEntropy_RNA_3_0',
 'Nuclei_Texture_SumEntropy_RNA_5_0',
 'Nuclei_Texture_SumVariance_AGP_10_0',
 'Nuclei_Texture_SumVariance_AGP_3_0',
 'Nuclei_Texture_SumVariance_AGP_5_0',
 'Nuclei_Texture_SumVariance_DNA_10_0',
 'Nuclei_Texture_SumVariance_DNA_3_0',
 'Nuclei_Texture_SumVariance_DNA_5_0',
 'Nuclei_Texture_SumVariance_ER_10_0',
 'Nuclei_Texture_SumVariance_ER_3_0',
 'Nuclei_Texture_SumVariance_ER_5_0',
 'Nuclei_Texture_SumVariance_Mito_10_0',
 'Nuclei_Texture_SumVariance_Mito_3_0',
 'Nuclei_Texture_SumVariance_Mito_5_0',
 'Nuclei_Texture_SumVariance_RNA_10_0',
 'Nuclei_Texture_SumVariance_RNA_3_0',
 'Nuclei_Texture_SumVariance_RNA_5_0',
 'Nuclei_Texture_Variance_AGP_10_0',
 'Nuclei_Texture_Variance_AGP_3_0',
 'Nuclei_Texture_Variance_AGP_5_0',
 'Nuclei_Texture_Variance_DNA_10_0',
 'Nuclei_Texture_Variance_DNA_3_0',
 'Nuclei_Texture_Variance_DNA_5_0',
 'Nuclei_Texture_Variance_ER_10_0',
 'Nuclei_Texture_Variance_ER_3_0',
 'Nuclei_Texture_Variance_ER_5_0',
 'Nuclei_Texture_Variance_Mito_10_0',
 'Nuclei_Texture_Variance_Mito_3_0',
 'Nuclei_Texture_Variance_Mito_5_0',
 'Nuclei_Texture_Variance_RNA_10_0',
 'Nuclei_Texture_Variance_RNA_3_0',
 'Nuclei_Texture_Variance_RNA_5_0'
</p>
</details>

### 2.3.1. Test Primary Key for `Image` Table

In [9]:
c.execute(
    """
    SELECT Image_Count_Cells, Image_Count_Cytoplasm, Image_Count_Nuclei
    FROM Image
    WHERE Image_Metadata_Plate = 24278 AND Image_Metadata_Site = '1' AND Image_Metadata_Well = 'a01'
    """
)
c.fetchall()

[(58.0, 58.0, 58.0)]

From above we can see there are 58 cells, 58 cytoplasm and 58 nuclei in the image of Plate 24278 Well `a01` in Site 1.

**This information is shared across all five channels.**

### 2.3.2. Test Primary Key for `Cells` Table

In [10]:
# Get the second combined key
c.execute(
    """
    SELECT TableNumber, ImageNumber
    FROM Image
    WHERE Image_Metadata_Plate = 24278 AND Image_Metadata_Site = '1' AND Image_Metadata_Well = 'a01'
    """
)
result = c.fetchall()
print(result)
tid, iid = result[0]

[('0702991209138712afb02ac7ea637f71', 1)]


`TableNumber` is actually a sting.

In [11]:
# Query in Cell Table
c.execute(
    """
    SELECT ObjectNumber
    FROM Cells
    WHERE TableNumber = '{}' AND ImageNumber = {}
    """.format(tid, iid)
)
result = c.fetchall()
print(result)

[(1,), (2,), (3,), (4,), (5,), (6,), (7,), (8,), (9,), (10,), (11,), (12,), (13,), (14,), (15,), (16,), (17,), (18,), (19,), (20,), (21,), (22,), (23,), (24,), (25,), (26,), (27,), (28,), (29,), (30,), (31,), (32,), (33,), (34,), (35,), (36,), (37,), (38,), (39,), (40,), (41,), (42,), (43,), (44,), (45,), (46,), (47,), (48,), (49,), (50,), (51,), (52,), (53,), (54,), (55,), (56,), (57,), (58,)]


As expected, there are 58 objects (`Image_Count_Cells` in the `Image` table).

We can crop rectangle or circular images using the following features:

- Cells_AreaShape_Center_X
- Cells_AreaShape_Center_X
- Cells_AreaShape_Center_Area
- Cells_AreaShape_MajorAxisLength
- Cells_AreaShape_MinorAxisLength
- Cells_AreaShape_MaximumRadius
- Cells_AreaShape_Orientation

This dataset also provides Zernike basis. I found it can be used in Computer Vision task in this [paper](https://onlinelibrary.wiley.com/doi/full/10.1111/j.1365-2818.2007.01799.x):

> Analogously, Zernike polynomials provide a 2D basis set which was originally designed to describe optical aberrations in lenses (Noll, 1976; Wang & Silva, 1980). These polynomials can also be used to break down images into contributions from lower and higher frequency optical patterns with no assumptions of periodicity (unlike 2D Fourier transforms), providing good numerical summaries of those images for computer vision tasks (Khotanzad & Hong, 1990). 

Some other interesting features are:

- Cells_Number_Object_Number
- Cells_Parent_Nuclei
- Cells_Neighbors_NumberOfNeighbors_Adjacent
- Cells_Neighbors_PercentTouching_Adjacent

In [12]:
# Query all 58 cells
c.execute(
    """
    SELECT Cells_AreaShape_Center_X, Cells_AreaShape_Center_Y, Cells_AreaShape_MajorAxisLength,
        Cells_AreaShape_MinorAxisLength, Cells_AreaShape_MaximumRadius, Cells_AreaShape_Orientation
    FROM Cells
    WHERE TableNumber = '{}' AND ImageNumber = {} AND ObjectNumber BETWEEN {} AND {};
    """.format(tid, iid, 1, 58)
)

result = c.fetchall()
df = pd.DataFrame(np.array(result),
                  columns=["CenterX", "CenterY", "MajorAxisLength",
                           "MinorAxisLength", "MaximumRadius", "Orientation"])
df.head()

,CenterX,CenterY,MajorAxisLength,MinorAxisLength,MaximumRadius,Orientation
0,43.0,0.0,70.397234,36.222035,21.023796,63.738571
1,619.0,28.0,97.565456,42.534725,20.000000,-34.521891
2,219.0,33.0,68.268046,49.118568,20.615528,-73.795204
3,265.0,32.0,75.198694,34.684289,17.691806,-28.019719
4,82.0,11.0,54.504681,40.508727,18.681542,-70.130892


## 3. Chemical Compound Matching

In this dataset, there are 30,616 compounds in total. 10,080 compounds came from the Molecular Libraries Small Molecule Repository (MLSMR), 2,260 were from Broad Institute bioactive compound collection, 269 were from the Molecular Libraries Program (MLP), and the other 18 051 were novel compounds from diversity-oriented synthesis.

We want to see if there are any intersection between this dataset and the one Chao and Moayad are using, especially RMI-FANCM.

In [23]:
df = pd.read_csv("./data/test/meta_data/chemical_annotations_smiles.csv")
print(df.shape)
df.head(7)

(30616, 11)


,BROAD_ID,CPD_NAME,CPD_NAME_TYPE,CPD_SAMPLE_ID,DOS_LIBRARY,SOURCE_NAME,CHEMIST_NAME,VENDOR_CATALOG_ID,CPD_SMILES,USERCOMMENT,CPD_CANONICAL_SMILES
0,BRD-A56675431-001-04-0,altizide,INN,SA82748,NaN,Prestwick Chemical Inc.,NaN,Prestw-721,NS(=O)(=O)c1cc2c(NC(CSCC=C)NS2(=O)=O)cc1Cl,NaN,C=CCSCC1Nc2cc(Cl)c(S(N)(=O)=O)cc2S(=O)(=O)N1
1,BRD-A51829654-001-01-4,"BRL-15,572",common,SA82481,NaN,Biomol International Inc.,NaN,AC-536,OC(CN1CCN(CC1)c1cccc(Cl)c1)C(c1ccccc1)c1ccccc1,NaN,OC(CN1CCN(c2cccc(Cl)c2)CC1)C(c1ccccc1)c1ccccc1
2,BRD-K04046242-001-03-6,equilin,primary-common,SA82922,NaN,Prestwick Chemical Inc.,NaN,Prestw-850,C[C@]12CC[C@H]3C(=CCc4cc(O)ccc34)[C@@H]1CCC2=O,NaN,C[C@]12CC[C@H]3C(=CCc4cc(O)ccc43)[C@@H]1CCC2=O
3,BRD-K16508793-001-01-8,diazepam,INN,SA59660,NaN,MicroSource Discovery Systems Inc.,NaN,1900003,CN1c2ccc(Cl)cc2C(=NCC1=O)c1ccccc1,NaN,CN1C(=O)CN=C(c2ccccc2)c2cc(Cl)ccc21
4,BRD-K09397065-001-01-6,SR 57227A,to-be-curated,SA82504,NaN,Biomol International Inc.,NaN,AC-561,NC1CCN(CC1)c1cccc(Cl)n1,NaN,NC1CCN(c2cccc(Cl)n2)CC1
5,BRD-K11927976-050-01-1,ER-27319,to-be-curated,SA792875,NaN,Tocris Bioscience,NaN,2471,Cc1ccc2c(c1C)n(CCCN)c1ccccc1c2=O,NaN,Cc1ccc2c(=O)c3ccccc3n(CCCN)c2c1C
6,BRD-K14282469-001-01-5,PAPP,primary-common,SA82523,NaN,Biomol International Inc.,NaN,AC-846,Nc1ccc(CCN2CCN(CC2)c2cccc(c2)C(F)(F)F)cc1,NaN,Nc1ccc(CCN2CCN(c3cccc(C(F)(F)F)c3)CC2)cc1


In [40]:
df_prospective = pd.read_csv('./data/test/pria_data/pria_prospective.csv')
print(df_prospective.shape)
df_prospective.head()

(25279, 5)


,Molecule,SMILES,Fingerprints,Keck_Pria_AS_Retest,Keck_Pria_Continuous
0,SMSSF-0612944,Cc1ccc(C(=O)Nc2ccc(Cc3ccc(NC(=O)c4ccc(C)cc4)cc...,0000000000000000000000000000000001000001000000...,0,5.602391
1,SMSSF-0595159,Cc1ccc(-c2csc(NC(=O)c3cccc(Br)c3)n2)c(C)c1,0000000000000001000000000000000001000000000000...,0,-3.537502
2,SMSSF-0595160,O=C(Nc1cc(Cl)cc(Cl)c1)c1ccc(C(=O)c2ccccc2)cc1,0000000001000000000000000000000000000000000000...,0,-8.542407
3,SMSSF-0595161,CCCCOc1ccc(C(=O)Nc2ccc(Cl)cc2C(=O)c2ccccc2)cc1,0000000001100100000000000000000001000000000000...,0,0.485958
4,SMSSF-0595162,CCCCOc1ccc(C(=O)Nc2ccc(-c3ccc(NC(=O)c4ccc(OCCC...,0000000000100100000000000000000001000000000000...,0,0.356097


In [51]:
intersect_prospective = set(df['CPD_CANONICAL_SMILES']) & set(df_prospective['SMILES'])
print("There are {} compounds also used in PriA-SSB prospective".format(len(intersect_prospective)))
intersect_prospective

There are 25 compounds also used in PriA-SSB prospective


{'C=CCN(CC=C)CC(O)Cn1c2ccccc2c2ccccc21',
 'CC(=O)Nc1cccc(O)c1',
 'CC(=O)c1ccc(NC(=O)c2ccc(CSc3ccccc3)o2)cc1',
 'CC(C)c1ccc2nc(N)sc2c1',
 'CCCCOC(=O)CSCC(=O)N1N=C(c2cccs2)CC1c1ccc(F)cc1',
 'CCCCOC(=O)c1ccc(O)cc1',
 'CCN1C(=O)c2cccc3cccc1c23',
 'CCOC(=O)c1c(C)[nH]c(C(=O)CSc2ccc(Cl)cc2)c1C',
 'CCc1nnc(NC(=O)c2ccc(S(=O)(=O)N3CCCc4ccccc43)cc2)o1',
 'CN(C)CCCN(C(=O)c1ccc2c(c1)CCCC2)c1nc2ccc(Cl)cc2s1',
 'CN(C)CCN(C(=O)C1CCCCC1)c1nc2cc3c(cc2s1)OCO3',
 'COC(=O)Cn1c(CN2CCc3ccccc3C2)nc2c1c(=O)n(C)c(=O)n2C',
 'COC(=O)c1ccsc1NC(=O)CCc1ccccc1',
 'COCCn1c(SCC(=O)Nc2cc(Cl)ccc2OC)nc2c(c1=O)SC(C)C2',
 'COc1cc(S(=O)(=O)N2CCN(c3ccccc3)CC2)ccc1Cl',
 'COc1ccc(-c2ccc(SCC(=O)N3CCOCC3)nn2)cc1OC',
 'Cc1cc(C(=O)Nc2cccnc2)no1',
 'Cc1cc(C(C)(C)C)cc(C)c1CC1=NCCN1',
 'Cc1ccc(Cn2ncc3c(N4CCN(c5ccccc5C)CC4)ncnc32)cc1',
 'O=C(CN1C(=O)c2ccccc2S1(=O)=O)N1CCN(c2ccccc2F)CC1',
 'O=C(CSc1ncnc2c1cnn2-c1ccc(F)cc1)NCC1CCCO1',
 'O=C(CSc1nnc(-c2ccccc2Cl)o1)N1CCCCC1',
 'O=C(Nc1nc2c(s1)CCCC2)c1cc(-c2ccccc2)nc2ccccc12',
 'O=C1c2ccccc

In [46]:
smiles_0 = pd.read_csv('./data/test/pria_data/pria_rmi_pcba_cv/file_0.csv')['SMILES']
smiles_1 = pd.read_csv('./data/test/pria_data/pria_rmi_pcba_cv/file_1.csv')['SMILES']
smiles_2 = pd.read_csv('./data/test/pria_data/pria_rmi_pcba_cv/file_2.csv')['SMILES']
smiles_3 = pd.read_csv('./data/test/pria_data/pria_rmi_pcba_cv/file_3.csv')['SMILES']
smiles_4 = pd.read_csv('./data/test/pria_data/pria_rmi_pcba_cv/file_4.csv')['SMILES']

In [54]:
intersect_others = set(df['CPD_CANONICAL_SMILES']) & (set(smiles_0) | set(smiles_1) | set(smiles_2) |
                                                      set(smiles_3) | set(smiles_4))
print("There are {} compounds also used in PriA-SSB AS, PriA-SSB FP, and RMI-FANCM FP".
      format(len(intersect_others)))
intersect_others

There are 13 compounds also used in PriA-SSB AS, PriA-SSB FP, and RMI-FANCM FP


{'CC(=O)OC1CN2CCC1CC2',
 'CC(C)SC(=N)N',
 'CCCCCC(=O)NC(=S)NC1C2CC3CC(C2)CC1C3',
 'CCCCCC=CCC=CCC=CCC=CCCCC(=O)NCCCC(=O)O',
 'CCCCCCCCC=CCCCCCCCC(=O)NCCO',
 'CCCCCCCCCCCCCCCC(=O)NCCO',
 'CCN(CC)C(=S)SSC(=S)N(CC)CC',
 'CCN1C(=O)C=CC1=O',
 'CCOC(=O)NCC(C)(C)CNC(=O)OCC',
 'CCSC(=N)N',
 'CN(C)CCCSC(=N)N',
 'CN1C2CCCC1CC(=O)C2',
 'CNC(=O)CSP(=S)(OC)OC'}